# Linguistic Fingerprints on Translation’s Lens: Jupyter notebook for April 15, 2019 presentation
April 15, 2019 presentation at the [Workshop on Digital Humanities to Preserve Knowledge and Cultural Heritage](https://cesta.stanford.edu/events/workshop-digital-humanities-preserve-knowledge-and-cultural-heritage)

Quinn Dombrowski, Yulia Ilchuk, Antonio Lenzo, J.D. Porter


The code in this notebook was used to generate the results in [slide 12 of the presentation deck](https://docs.google.com/presentation/d/1JjIh6T1ZPtzGHzTq2zTSfKPyUVmIZfV3TTbk5tEQ0w4/edit#slide=id.g57e206c36b_0_21).

v. 1.0, published April 15, 2019


## Suggested citation

Quinn Dombrowski. "Linguistic Fingerprints on Translation’s Lens: Jupyter notebook for April 15, 2019 presentation". v. 1.0, published April 15, 2019.

## Corpus
For our source material, we collected modern (mid-20th/21st century) short stories translated from Italian, Portuguese, Russian, and Spanish, as well as original stories in English. There were 20 stories representing each language. The corpus isn't republishable due to copyright, but you can find the [metadata about the texts in this CSV](litlab_translations_2019-04-15_corpus_metadata.csv). Each text file contained only the text of the short story; title and author information, trainslator footnotes, etc. were stripped out.

## 1. Basic setup
- Make sure Spacy is installed (e.g. run `pip install spacy` in the Terminal) before running this notebook
- Put each language's texts in its own folder
- In each folder, using the Terminal, run `cat * > all_ru.txt` (replacing *ru* w/ 2-letter abbreviation for the language in question)
- Create a new folder for all these new files, and put them in that directory. Specify the path in the first code block, below

In [1]:
#os is used for things like changing directories and listing files
import os
#io is used for opening and writing files
import io
#itertools is used for some of the iterative code
from itertools import chain
#glob is used to find all the pathnames matching a specified pattern (here, all text files)
import glob

# used for some text matching
import re

# ntlk's sentence tokenizer was easier to use as a first step in sentence length
from nltk import sent_tokenize

# spacy is used for most of the NLP
import spacy
nlp = spacy.load('en_core_web_sm')
from spacy.tokenizer import Tokenizer
from spacy.lemmatizer import Lemmatizer
lemmatizer = Lemmatizer()
from spacy.matcher import Matcher
matcher = Matcher(nlp.vocab)

# scipy stats is used to calculate Fisher's exact
import scipy.stats


#Full path to directory with allru.txt, allen.txt, alles.txt, etc.
sourcefiledirectory = '/Users/qad/Documents/translationcorpus/all'


#Changing the directory to where you've stored the source texts, so you can open them in later code blocks
os.chdir(sourcefiledirectory)

## 2. Get the word count for each corpus
This creates a derivative file, wordcount_XX.txt (with XX replaced by the two-letter language code) with the word count for each language corpus, and prints the results below. You can copy and paste the values printed below into Excel for getting your values for Fisher's exact.

In [115]:
for filename in os.listdir(sourcefiledirectory):
    if filename.startswith("all"):
        outname = filename.replace('all', 'wordcount')
        with open(outname, 'w') as out:
            f = open(filename, 'r')
            text = f.read()
            words = 0
            for wordcount in text.split(" "):
                words += 1
            print(filename)
            print((str(words)))
            out.write(str(words))

all_ru.txt
158965
all_en.txt
104309
all_pt.txt
69059
all_it.txt
72917
all_es.txt
71378


## 3. Calculating lemma/word count ratio
To get the lemma/word count ratio (on the right side of slide 12), we need to count all the unique lemmas, and compare that to the total word count in (2),  above.

### Creating lemma file
The code block below creates a file, lemmas_XX.txt, that contains the lemma value for each word in the text.

In [ ]:
word_count = 0
for filename in os.listdir(sourcefiledirectory):
    if filename.startswith("all"):
        
        f = open(filename, 'r')
        text = f.read()
        lines = sent_tokenize(text)
        outname = filename.replace('all', 'lemmas')
        with open(outname, 'w') as out:
            for line in (lines):
                annotated_line = nlp(line)
                for token in annotated_line:
                    out.write(token.lemma_)
                    out.write(" ")

### Unique lemmas
This creates a file wiht the total count of unique lemmas in each corpus, and also prints those values below.

In [136]:
#count & print to file unique lemmas
for filename in os.listdir(sourcefiledirectory):
    if filename.startswith("lemmas"):
        with open(filename, 'r') as f:
            lines = f.read().splitlines()
            uniques = set()
            for line in lines:
                uniques |= set(line.split())
            outname = filename.replace('lemmas', 'uniquelemmacount')
            with open(outname, 'w') as out:
                out.write(str(len(uniques)))
            print(filename)
            print(len(uniques))

lemmas__pt.txt
7453
lemmas__it.txt
7795
lemmas__es.txt
6618
lemmas__ru.txt
11269
lemmas__en.txt
9125


### Calculating lemma / token ratio
In Excel (or your favorite software), divide the unique lemma count values with the total word count for each language. The values are in the *Lemmas-tokens* tab of the [working spreadsheet for the presentation](litlab_translations_2019-04-15_fishers_exact_values.xlsx).

## 4. Calculating probability of statistically significant divergence from English for various traits

The traits below (which appear in the table on slide 12) were selected as possible features for distinguishing Russian translations from originally-English texts, and/or were referenced in previous work on translated texts as distinguishing features.

For each trait, we count the occurrences in each of the corpora.

To determine which are statistically significance, we use Fisher's exact test to compare the observed values for the trait to the "null-hypothesis" value, which we base on the observed rate in English. We use the rate the trait occurs in English, and multiply it by the word count for each non-English corpus, to get the "expected" rate for that language if it were to not differ from English more than chance.

Fisher's exact test is calculated with four values, denoted \[a,b\]\[c,d\].
* a: the observed rate for the non-English language
* b: the not-observed rate for the non-English language (word count - a)
* c: the "null-hypothesis" expected value (rate in English x word count)
* d: the "null-hypothesis" expected not-observed value (word count - c)

The [working Excel spreadsheet for the presentation](litlab_translations_2019-04-15_fishers_exact_values.xlsx) has formulas set up for all of these, once you've put in the word count and observed rates for each language (including English).

Because Fisher's exact is looking for the probability that the observed values don't differ from the null-hypothesis values except by chance, **low** values indicate a higher probability that the language has an impact on the occurrence rate of the trait in question.

### 4.1 Determiner count
Russian has many fewer determiners (e.g. no definite / indefinite article), so where determiners exist in translations, they are largely inserted by translators. 

The first code block writes out the spacy IDs for each determiner to a temporary text file, then counts the lines in that text file and writes that count to a file, DET_XX.txt for each language.

The value in DET_XX.txt becomes "a" for calculating Fisher's exact, in the second code block. Value "b" is the total word count. The values of "c" and "d" are calculated in the Excel spreadsheet using the rate for English, multiplied by the word count of the non-English corpus.

In [67]:
#determiners
pattern = [{"POS": "DET", "TAG": "DT"}]

matcher.add("DET", None, pattern)
for filename in os.listdir(sourcefiledirectory):
    if filename.startswith("all"):
        
        f = open(filename, 'r')
        text = f.read()
        lines = sent_tokenize(text)
        outname = filename.replace('all', 'tmp_')
        with open(outname, 'w') as out:
            print(filename)
            for index,line in enumerate(lines):
                annotated_line = nlp(line)
                matches = matcher(annotated_line)
                for match_id, start, end in matches:
                    string_id = nlp.vocab.strings[match_id]  # Get string representation
                    span = annotated_line[start:end]  # The matched span
                    print(match_id, string_id, start, end, span.text)
                    out.write(str(match_id))
                    out.write("\n")
for filename in os.listdir(sourcefiledirectory):
    if filename.startswith("tmp_"):
        with open(filename, 'r') as f:
            outname = filename.replace('tmp_', 'DET_')
            num_lines = 0
            with open(outname, 'w') as out:
                for line in f:
                    num_lines += 1
                print(filename)
                print(num_lines)
                out.write(str(num_lines))
matcher.remove("DET")

all_ru.txt
90 DET 3 4 a
90 DET 9 10 the
90 DET 12 13 the
90 DET 12 13 an
90 DET 20 21 some
90 DET 0 1 That
90 DET 8 9 the
90 DET 16 17 the
90 DET 11 12 the
90 DET 0 1 That
90 DET 1 2 the
90 DET 4 5 a
90 DET 3 4 an
90 DET 8 9 the
90 DET 0 1 That
90 DET 6 7 a
90 DET 9 10 a
90 DET 4 5 the
90 DET 0 1 The
90 DET 0 1 The
90 DET 7 8 the
90 DET 13 14 a
90 DET 0 1 No
90 DET 12 13 the
90 DET 7 8 a
90 DET 19 20 a
90 DET 0 1 A
90 DET 10 11 the
90 DET 13 14 the
90 DET 7 8 the
90 DET 18 19 a
90 DET 3 4 an
90 DET 6 7 a
90 DET 2 3 the
90 DET 17 18 a
90 DET 25 26 the
90 DET 31 32 a
90 DET 37 38 an
90 DET 42 43 a
90 DET 46 47 the
90 DET 8 9 a
90 DET 0 1 This
90 DET 4 5 a
90 DET 10 11 every
90 DET 4 5 no
90 DET 5 6 a
90 DET 2 3 the
90 DET 5 6 the
90 DET 10 11 the
90 DET 17 18 a
90 DET 29 30 the
90 DET 6 7 the
90 DET 13 14 a
90 DET 3 4 the
90 DET 0 1 The
90 DET 2 3 a
90 DET 10 11 the
90 DET 4 5 no
90 DET 11 12 the
90 DET 17 18 The
90 DET 11 12 no
90 DET 14 15 the
90 DET 12 13 a
90 DET 30 31 the
90 DET 4 5

90 DET 3 4 a
90 DET 13 14 a
90 DET 21 22 the
90 DET 6 7 any
90 DET 0 1 The
90 DET 4 5 the
90 DET 0 1 A
90 DET 4 5 an
90 DET 0 1 The
90 DET 5 6 an
90 DET 0 1 A
90 DET 6 7 the
90 DET 1 2 that
90 DET 4 5 a
90 DET 12 13 the
90 DET 17 18 the
90 DET 22 23 a
90 DET 27 28 the
90 DET 31 32 a
90 DET 36 37 a
90 DET 2 3 no
90 DET 9 10 the
90 DET 15 16 the
90 DET 20 21 the
90 DET 4 5 the
90 DET 9 10 that
90 DET 14 15 The
90 DET 24 25 any
90 DET 28 29 the
90 DET 35 36 the
90 DET 4 5 the
90 DET 2 3 a
90 DET 10 11 a
90 DET 14 15 the
90 DET 4 5 a
90 DET 4 5 another
90 DET 0 1 That
90 DET 5 6 a
90 DET 13 14 the
90 DET 2 3 the
90 DET 7 8 the
90 DET 13 14 the
90 DET 19 20 a
90 DET 0 1 The
90 DET 12 13 the
90 DET 21 22 the
90 DET 28 29 the
90 DET 2 3 the
90 DET 8 9 the
90 DET 13 14 the
90 DET 1 2 the
90 DET 7 8 the
90 DET 18 19 the
90 DET 21 22 the
90 DET 28 29 the
90 DET 5 6 the
90 DET 8 9 the
90 DET 28 29 the
90 DET 31 32 the
90 DET 34 35 the
90 DET 3 4 the
90 DET 19 20 the
90 DET 3 4 the
90 DET 7 8 some

90 DET 3 4 this
90 DET 24 25 a
90 DET 27 28 a
90 DET 0 1 The
90 DET 9 10 a
90 DET 3 4 that
90 DET 13 14 a
90 DET 4 5 the
90 DET 8 9 the
90 DET 0 1 The
90 DET 4 5 the
90 DET 11 12 the
90 DET 12 13 any
90 DET 5 6 a
90 DET 8 9 this
90 DET 0 1 The
90 DET 6 7 the
90 DET 5 6 the
90 DET 11 12 the
90 DET 14 15 a
90 DET 2 3 the
90 DET 5 6 the
90 DET 18 19 some
90 DET 5 6 that
90 DET 13 14 the
90 DET 3 4 this
90 DET 2 3 that
90 DET 0 1 The
90 DET 4 5 the
90 DET 7 8 an
90 DET 12 13 the
90 DET 16 17 some
90 DET 18 19 the
90 DET 20 21 a
90 DET 3 4 a
90 DET 1 2 a
90 DET 4 5 this
90 DET 5 6 the
90 DET 4 5 the
90 DET 5 6 some
90 DET 8 9 an
90 DET 12 13 no
90 DET 18 19 the
90 DET 7 8 a
90 DET 5 6 the
90 DET 0 1 Some
90 DET 2 3 the
90 DET 2 3 an
90 DET 6 7 the
90 DET 7 8 an
90 DET 18 19 the
90 DET 21 22 the
90 DET 1 2 that
90 DET 6 7 the
90 DET 5 6 the
90 DET 6 7 a
90 DET 12 13 some
90 DET 14 15 the
90 DET 4 5 a
90 DET 7 8 a
90 DET 12 13 any
90 DET 15 16 any
90 DET 4 5 a
90 DET 8 9 the
90 DET 16 17 anot

90 DET 13 14 the
90 DET 0 1 The
90 DET 3 4 no
90 DET 6 7 the
90 DET 10 11 the
90 DET 0 1 The
90 DET 12 13 The
90 DET 17 18 the
90 DET 11 12 the
90 DET 21 22 the
90 DET 24 25 a
90 DET 0 1 The
90 DET 0 1 The
90 DET 0 1 The
90 DET 3 4 the
90 DET 7 8 an
90 DET 15 16 all
90 DET 0 1 Every
90 DET 5 6 a
90 DET 10 11 The
90 DET 18 19 the
90 DET 6 7 a
90 DET 12 13 a
90 DET 16 17 the
90 DET 4 5 a
90 DET 14 15 A
90 DET 20 21 the
90 DET 0 1 The
90 DET 1 2 those
90 DET 6 7 the
90 DET 11 12 any
90 DET 22 23 the
90 DET 25 26 the
90 DET 3 4 the
90 DET 7 8 the
90 DET 23 24 the
90 DET 2 3 the
90 DET 10 11 the
90 DET 18 19 the
90 DET 0 1 The
90 DET 2 3 a
90 DET 1 2 a
90 DET 0 1 That
90 DET 8 9 the
90 DET 7 8 the
90 DET 1 2 the
90 DET 10 11 a
90 DET 0 1 That
90 DET 0 1 A
90 DET 1 2 that
90 DET 10 11 a
90 DET 14 15 the
90 DET 5 6 the
90 DET 12 13 the
90 DET 3 4 the
90 DET 17 18 the
90 DET 13 14 a
90 DET 26 27 the
90 DET 0 1 All
90 DET 23 24 the
90 DET 3 4 the
90 DET 9 10 a
90 DET 15 16 the
90 DET 2 3 the
90

90 DET 8 9 That
90 DET 0 1 That
90 DET 2 3 a
90 DET 17 18 no
90 DET 0 1 That
90 DET 19 20 the
90 DET 34 35 The
90 DET 42 43 a
90 DET 12 13 the
90 DET 18 19 an
90 DET 1 2 a
90 DET 5 6 a
90 DET 12 13 a
90 DET 0 1 A
90 DET 6 7 a
90 DET 14 15 the
90 DET 18 19 a
90 DET 3 4 the
90 DET 10 11 the
90 DET 2 3 a
90 DET 6 7 the
90 DET 9 10 a
90 DET 15 16 a
90 DET 7 8 an
90 DET 0 1 The
90 DET 5 6 a
90 DET 4 5 the
90 DET 13 14 a
90 DET 13 14 the
90 DET 2 3 the
90 DET 12 13 the
90 DET 39 40 the
90 DET 42 43 this
90 DET 2 3 the
90 DET 19 20 a
90 DET 12 13 the
90 DET 16 17 a
90 DET 23 24 the
90 DET 26 27 some
90 DET 4 5 a
90 DET 3 4 that
90 DET 2 3 a
90 DET 4 5 the
90 DET 0 1 That
90 DET 0 1 That
90 DET 5 6 the
90 DET 8 9 The
90 DET 13 14 the
90 DET 5 6 the
90 DET 8 9 the
90 DET 10 11 a
90 DET 0 1 The
90 DET 5 6 the
90 DET 0 1 The
90 DET 11 12 a
90 DET 2 3 a
90 DET 16 17 the
90 DET 3 4 an
90 DET 15 16 the
90 DET 21 22 a
90 DET 4 5 any
90 DET 5 6 the
90 DET 10 11 all
90 DET 4 5 the
90 DET 7 8 a
90 DET 1

...

90 DET 7 8 this
90 DET 10 11 a
90 DET 7 8 a
90 DET 0 1 The
90 DET 9 10 the
90 DET 15 16 all
90 DET 10 11 the
90 DET 18 19 a
90 DET 8 9 the
90 DET 7 8 the
90 DET 16 17 the
90 DET 7 8 the
90 DET 24 25 a
90 DET 32 33 a
90 DET 10 11 an
90 DET 21 22 a
90 DET 4 5 every
90 DET 12 13 no
90 DET 20 21 the
90 DET 26 27 the
90 DET 5 6 the
90 DET 0 1 A
90 DET 7 8 a
90 DET 12 13 the
90 DET 19 20 the
90 DET 8 9 the
90 DET 10 11 the
90 DET 0 1 No
90 DET 5 6 a
90 DET 4 5 a
90 DET 3 4 a
90 DET 4 5 a
90 DET 11 12 That
90 DET 2 3 the
90 DET 5 6 that
90 DET 2 3 this
90 DET 9 10 the
90 DET 15 16 a
90 DET 29 30 the
90 DET 33 34 the
90 DET 36 37 a
90 DET 5 6 the
90 DET 8 9 the
90 DET 14 15 the
90 DET 20 21 a
90 DET 31 32 the
90 DET 42 43 the
90 DET 4 5 the
90 DET 6 7 the
90 DET 13 14 a
90 DET 2 3 the
90 DET 5 6 the
90 DET 8 9 the
90 DET 17 18 all
90 DET 20 21 the
90 DET 32 33 the
90 DET 45 46 the
90 DET 48 49 a
90 DET 2 3 the
90 DET 3 4 no
90 DET 8 9 no
90 DET 31 32 the
90 DET 13 14 the
90 DET 16 17 a
90 DET 

90 DET 1 2 that
90 DET 19 20 both
90 DET 1 2 this
90 DET 16 17 the
90 DET 3 4 an
90 DET 6 7 the
90 DET 18 19 a
90 DET 22 23 a
90 DET 40 41 the
90 DET 52 53 a
90 DET 5 6 a
90 DET 5 6 a
90 DET 11 12 the
90 DET 18 19 a
90 DET 0 1 This
90 DET 8 9 every
90 DET 11 12 a
90 DET 6 7 The
90 DET 2 3 this
90 DET 5 6 a
90 DET 10 11 that
90 DET 17 18 a
90 DET 13 14 the
90 DET 12 13 a
90 DET 0 1 This
90 DET 5 6 the
90 DET 13 14 the
90 DET 29 30 all
90 DET 0 1 The
90 DET 5 6 a
90 DET 13 14 a
90 DET 9 10 a
90 DET 4 5 an
90 DET 14 15 a
90 DET 25 26 a
90 DET 1 2 the
90 DET 10 11 the
90 DET 0 1 The
90 DET 15 16 a
90 DET 18 19 the
90 DET 4 5 a
90 DET 8 9 a
90 DET 3 4 a
90 DET 3 4 othe
90 DET 5 6 a
90 DET 3 4 a
90 DET 7 8 this
90 DET 10 11 the
90 DET 4 5 an
90 DET 8 9 the
90 DET 9 10 this
90 DET 24 25 this
90 DET 38 39 a
90 DET 5 6 an
90 DET 8 9 the
90 DET 12 13 the
90 DET 22 23 a
90 DET 29 30 an
90 DET 5 6 the
90 DET 1 2 a
90 DET 10 11 the
90 DET 12 13 a
90 DET 26 27 these
90 DET 32 33 the
90 DET 6 7 a
90 

90 DET 7 8 every
90 DET 12 13 the
90 DET 18 19 the
90 DET 28 29 the
90 DET 44 45 all
90 DET 50 51 this
90 DET 59 60 the
90 DET 9 10 the
90 DET 3 4 a
90 DET 12 13 each
90 DET 15 16 a
90 DET 1 2 the
90 DET 11 12 a
90 DET 22 23 the
90 DET 25 26 the
90 DET 31 32 an
90 DET 0 1 This
90 DET 13 14 the
90 DET 1 2 this
90 DET 14 15 a
90 DET 4 5 the
90 DET 9 10 the
90 DET 1 2 the
90 DET 6 7 the
90 DET 14 15 the
90 DET 6 7 the
90 DET 6 7 this
90 DET 9 10 a
90 DET 6 7 this
90 DET 0 1 The
90 DET 15 16 this
90 DET 5 6 the
90 DET 20 21 the
90 DET 10 11 these
90 DET 5 6 the
90 DET 0 1 That
90 DET 26 27 a
90 DET 24 25 the
90 DET 27 28 a
90 DET 38 39 the
90 DET 43 44 a
90 DET 53 54 a
90 DET 60 61 the
90 DET 67 68 a
90 DET 73 74 the
90 DET 9 10 the
90 DET 15 16 the
90 DET 9 10 the
90 DET 4 5 those
90 DET 8 9 the
90 DET 15 16 the
90 DET 19 20 the
90 DET 27 28 a
90 DET 31 32 the
90 DET 36 37 the
90 DET 40 41 the
90 DET 46 47 a
90 DET 49 50 the
90 DET 4 5 the
90 DET 7 8 the
90 DET 14 15 the
90 DET 22 23 the


90 DET 7 8 an
90 DET 26 27 a
90 DET 31 32 a
90 DET 9 10 the
90 DET 6 7 the
90 DET 13 14 a
90 DET 16 17 the
90 DET 24 25 the
90 DET 27 28 the
90 DET 7 8 all
90 DET 14 15 the
90 DET 25 26 an
90 DET 27 28 the
90 DET 30 31 a
90 DET 34 35 the
90 DET 37 38 an
90 DET 41 42 the
90 DET 44 45 the
90 DET 8 9 the
90 DET 12 13 a
90 DET 14 15 the
90 DET 2 3 the
90 DET 15 16 a
90 DET 18 19 this
90 DET 0 1 The
90 DET 3 4 the
90 DET 0 1 All
90 DET 7 8 a
90 DET 0 1 Every
90 DET 7 8 a
90 DET 16 17 the
90 DET 19 20 that
90 DET 22 23 the
90 DET 12 13 the
90 DET 15 16 a
90 DET 0 1 Every
90 DET 2 3 the
90 DET 8 9 a
90 DET 36 37 the
90 DET 0 1 The
90 DET 8 9 a
90 DET 14 15 the
90 DET 5 6 the
90 DET 11 12 the
90 DET 25 26 the
90 DET 28 29 a
90 DET 12 13 a
90 DET 0 1 The
90 DET 4 5 the
90 DET 15 16 a
90 DET 14 15 the
90 DET 0 1 This
90 DET 10 11 the
90 DET 14 15 the
90 DET 17 18 the
90 DET 1 2 the
90 DET 6 7 the
90 DET 10 11 the
90 DET 13 14 the
90 DET 26 27 the
90 DET 29 30 the
90 DET 4 5 the
90 DET 4 5 the
90

90 DET 14 15 a
90 DET 1 2 that
90 DET 8 9 a
90 DET 33 34 the
90 DET 0 1 The
90 DET 7 8 those
90 DET 17 18 the
90 DET 11 12 a
90 DET 14 15 the
90 DET 35 36 the
90 DET 3 4 Those
90 DET 5 6 the
90 DET 4 5 some
90 DET 6 7 the
90 DET 2 3 a
90 DET 8 9 a
90 DET 3 4 that
90 DET 6 7 this
90 DET 1 2 The
90 DET 17 18 the
90 DET 20 21 the
90 DET 1 2 the
90 DET 5 6 another
90 DET 21 22 a
90 DET 25 26 all
90 DET 3 4 The
90 DET 2 3 the
90 DET 3 4 the
90 DET 4 5 the
90 DET 21 22 the
90 DET 24 25 the
90 DET 29 30 the
90 DET 0 1 The
90 DET 12 13 the
90 DET 0 1 that
90 DET 5 6 the
90 DET 4 5 the
90 DET 0 1 These
90 DET 14 15 the
90 DET 7 8 the
90 DET 10 11 the
90 DET 20 21 a
90 DET 4 5 those
90 DET 7 8 the
90 DET 0 1 The
90 DET 2 3 the
90 DET 10 11 the
90 DET 25 26 the
90 DET 34 35 the
90 DET 8 9 the
90 DET 2 3 the
90 DET 13 14 the
90 DET 20 21 some
90 DET 43 44 the
90 DET 10 11 the
90 DET 5 6 any
90 DET 21 22 a
90 DET 14 15 those
90 DET 17 18 those
90 DET 20 21 an
90 DET 0 1 That
90 DET 2 3 the
90 DET 6

In [102]:
#Fisher's exact for determiners

print('determiners')
print('e.g. a, the, that')

#ru
oddsratio, pvalue = scipy.stats.fisher_exact([[17495, 141470], [16685, 142280]])
print('ru')
print(pvalue)

#pt
oddsratio, pvalue = scipy.stats.fisher_exact([[8555, 60504], [7248, 61811]])
print('pt')
print(pvalue)

#it
oddsratio, pvalue = scipy.stats.fisher_exact([[8627, 64290], [7653, 65264]])
print('it')
print(pvalue)

#es
oddsratio, pvalue = scipy.stats.fisher_exact([[7420, 63958], [7492, 63886]])
print('es')
print(pvalue)

determiners
e.g. a, the, that
ru
3.6198136985402884e-06
pt
2.310453535701314e-28
it
5.827055539147346e-16
es
0.5389557177465378


### 4.2 Gerund use
Russian more frequently uses constructions like "*Going* outside, she saw..." so we wanted to see if there were more constructions with gerund without an auxiliary verb beforehand.

We also wanted to look at gerund usage *with* auxiliary verbs, because that could reflect decisions on how to translate Russian aspect.

For gerunds with auxiliaries, we used word count / 2 (rather than word count) because it's a bigram, meaning the maximum possible value would then need to be word count divided in half (for a text that contained, for instance, nothing but repetitions of "was reading").

#### 4.2.1 Gerund w/o aux

In [3]:
#gerund/present particle w/o aux
pattern = [{"DEP": {"NOT_IN": ["aux"]}}, {"TAG": "VBG"}]

matcher.add("gerund", None, pattern)
for filename in os.listdir(sourcefiledirectory):
    if filename.startswith("all"):
        
        f = open(filename, 'r')
        text = f.read()
        lines = sent_tokenize(text)
        outname = filename.replace('all', 'tmp_')
        with open(outname, 'w') as out:
            print(filename)
            for index,line in enumerate(lines):
                annotated_line = nlp(line)
                matches = matcher(annotated_line)
                for match_id, start, end in matches:
                    string_id = nlp.vocab.strings[match_id]  # Get string representation
                    span = annotated_line[start:end]  # The matched span
                    print(match_id, string_id, start, end, span.text)
                    out.write(str(match_id))
                    out.write("\n")
for filename in os.listdir(sourcefiledirectory):
    if filename.startswith("tmp_"):
        with open(filename, 'r') as f:
            outname = filename.replace('tmp_', 'gerund_')
            num_lines = 0
            with open('outname', 'w') as out:
                for line in f:
                    num_lines += 1
                print(filename)
                print(num_lines)
                out.write(str(num_lines))
matcher.remove("gerund")

all_ru.txt
12539000998014861558 gerund 7 9 

Leaving
12539000998014861558 gerund 17 19 not waiting
12539000998014861558 gerund 2 4 just lying
12539000998014861558 gerund 14 16 , disintegrating
12539000998014861558 gerund 10 12 , snoring
12539000998014861558 gerund 9 11 keep yawning
12539000998014861558 gerund 11 13 and nodding
12539000998014861558 gerund 9 11 still sleeping
12539000998014861558 gerund 17 19 , exhaling
12539000998014861558 gerund 11 13 n’t sleeping
12539000998014861558 gerund 40 42 , forming
12539000998014861558 gerund 5 7 , drawing
12539000998014861558 gerund 0 2 This drawing
12539000998014861558 gerund 3 5 fastidiously interrogating
12539000998014861558 gerund 10 12 the drying
12539000998014861558 gerund 12 14 , kneading
12539000998014861558 gerund 0 2 Without glancing
12539000998014861558 gerund 2 4 like being
12539000998014861558 gerund 2 4 , spitting
12539000998014861558 gerund 9 11 a screaming
12539000998014861558 gerund 11 13 start howling
12539000998014861558 ge

12539000998014861558 gerund 12 14 already lying
12539000998014861558 gerund 12 14 even thinking
12539000998014861558 gerund 19 21 an aging
12539000998014861558 gerund 8 10 , steadying
12539000998014861558 gerund 15 17 no burning
12539000998014861558 gerund 0 2 After crossing
12539000998014861558 gerund 12 14 from watching
12539000998014861558 gerund 12 14 , muttering
12539000998014861558 gerund 23 25 start moving
12539000998014861558 gerund 3 5 even thinking
12539000998014861558 gerund 4 6 in thinking
12539000998014861558 gerund 4 6 -affirming
12539000998014861558 gerund 6 8 curses coming
12539000998014861558 gerund 5 7 you doing
12539000998014861558 gerund 16 18 without answering
12539000998014861558 gerund 27 29 a cooling
12539000998014861558 gerund 2 4 you trying
12539000998014861558 gerund 10 12 all going
12539000998014861558 gerund 22 24 , shaking
12539000998014861558 gerund 23 25 , threatening
12539000998014861558 gerund 3 5 you going
12539000998014861558 gerund 4 6 one asking
12

12539000998014861558 gerund 8 10 bother asking
12539000998014861558 gerund 6 8 crowd filling
12539000998014861558 gerund 6 8 few remaining
12539000998014861558 gerund 14 16 of waiting
12539000998014861558 gerund 5 7 hours killing
12539000998014861558 gerund 8 10 , letting
12539000998014861558 gerund 3 5 , touching
12539000998014861558 gerund 24 26 we going
12539000998014861558 gerund 2 4 -looking
12539000998014861558 gerund 6 8 and drinking
12539000998014861558 gerund 9 11 , turning
12539000998014861558 gerund 7 9 though straining
12539000998014861558 gerund 16 18 , walking
12539000998014861558 gerund 5 7 someone cracking
12539000998014861558 gerund 10 12 , pressing
12539000998014861558 gerund 5 7 , looping
12539000998014861558 gerund 7 9 when tailing
12539000998014861558 gerund 13 15 Annie going
12539000998014861558 gerund 19 21 windows reaching
12539000998014861558 gerund 3 5 all moving
12539000998014861558 gerund 2 4 lock barring
12539000998014861558 gerund 3 5 , moving
125390009980

12539000998014861558 gerund 27 29 , snaking
12539000998014861558 gerund 32 34 the falling
12539000998014861558 gerund 4 6 still smoldering
12539000998014861558 gerund 12 14 started cooling
12539000998014861558 gerund 7 9 despite having
12539000998014861558 gerund 3 5 finished giving
12539000998014861558 gerund 6 8 enjoyed talking
12539000998014861558 gerund 3 5 of having
12539000998014861558 gerund 9 11 , stuttering
12539000998014861558 gerund 12 14 , resuming
12539000998014861558 gerund 13 15 considering acquiring
12539000998014861558 gerund 6 8 , producing
12539000998014861558 gerund 11 13 , assuming
12539000998014861558 gerund 3 5 n’t looking
12539000998014861558 gerund 34 36 than getting
12539000998014861558 gerund 4 6 a practicing
12539000998014861558 gerund 15 17 for acclimatizing
12539000998014861558 gerund 18 20 the leading
12539000998014861558 gerund 22 24 include replenishing
12539000998014861558 gerund 28 30 , restoring
12539000998014861558 gerund 7 9 experience working
1253

12539000998014861558 gerund 2 4 stopped resisting
12539000998014861558 gerund 28 30 nurse administering
12539000998014861558 gerund 16 18 himself hanging
12539000998014861558 gerund 23 25 pipes searing
12539000998014861558 gerund 9 11 , looking
12539000998014861558 gerund 25 27 , brushing
12539000998014861558 gerund 30 32 and straightening
12539000998014861558 gerund 6 8 , chafing
12539000998014861558 gerund 10 12 , pulling
12539000998014861558 gerund 3 5 door using
12539000998014861558 gerund 8 10 , honing
12539000998014861558 gerund 1 3 ’m waiting
12539000998014861558 gerund 3 5 , turning
12539000998014861558 gerund 8 10 , saturating
12539000998014861558 gerund 10 12 their stinking
12539000998014861558 gerund 10 12 icicles hanging
12539000998014861558 gerund 5 7 ’re going
12539000998014861558 gerund 15 17 clothes soaking
12539000998014861558 gerund 4 6 in sagging
12539000998014861558 gerund 10 12 , breathing
12539000998014861558 gerund 11 13 overcoat trailing
12539000998014861558 ger

...

12539000998014861558 gerund 14 16 him stirring
12539000998014861558 gerund 8 10 ’s going
12539000998014861558 gerund 14 16 without pausing
12539000998014861558 gerund 19 21 , looking
12539000998014861558 gerund 2 4 ’re being
12539000998014861558 gerund 0 2 Without thinking
12539000998014861558 gerund 19 21 considered saying
12539000998014861558 gerund 23 25 and going
12539000998014861558 gerund 13 15 if muttering
12539000998014861558 gerund 5 7 not thinking
12539000998014861558 gerund 33 35 between talking
12539000998014861558 gerund 37 39 and talking
12539000998014861558 gerund 6 8 not thinking
12539000998014861558 gerund 18 20 , stretching
12539000998014861558 gerund 6 8 , laughing
12539000998014861558 gerund 4 6 in remembering
12539000998014861558 gerund 19 21 in seeing
12539000998014861558 gerund 14 16 my ensuing
12539000998014861558 gerund 9 11 when referring
12539000998014861558 gerund 7 9 continued talking
12539000998014861558 gerund 15 17 
Having
12539000998014861558 gerund 12 

12539000998014861558 gerund 16 18 him gliding
12539000998014861558 gerund 39 41 him carrying
12539000998014861558 gerund 4 6 figure looking
12539000998014861558 gerund 16 18 beautiful smiling
12539000998014861558 gerund 11 13 child screaming
12539000998014861558 gerund 18 20 , walking
12539000998014861558 gerund 8 10 , lulling
12539000998014861558 gerund 17 19 considered going
12539000998014861558 gerund 4 6 , fearing
12539000998014861558 gerund 13 15 , revealing
12539000998014861558 gerund 35 37 , singing
12539000998014861558 gerund 0 2 Her smiling
12539000998014861558 gerund 15 17 , alighting
12539000998014861558 gerund 1 3 , perceiving
12539000998014861558 gerund 4 6 , trusting
12539000998014861558 gerund 7 9 and going
12539000998014861558 gerund 5 7 , warming
12539000998014861558 gerund 3 5 , clutching
12539000998014861558 gerund 3 5 of falling
12539000998014861558 gerund 21 23 the pounding
12539000998014861558 gerund 14 16 of waiting
12539000998014861558 gerund 18 20 or giving
125

12539000998014861558 gerund 30 32 of lodging
12539000998014861558 gerund 21 23 , strewing
12539000998014861558 gerund 9 11 not knowing
12539000998014861558 gerund 12 14 , knocking
12539000998014861558 gerund 20 22 though calling
12539000998014861558 gerund 36 38 the burning
12539000998014861558 gerund 9 11 , splashing
12539000998014861558 gerund 22 24 rivers sliding
12539000998014861558 gerund 13 15 storm raging
12539000998014861558 gerund 2 4 pearls raining
12539000998014861558 gerund 16 18 likewise maintaining
12539000998014861558 gerund 14 16 grass floating
12539000998014861558 gerund 23 25 mirrors wearing
12539000998014861558 gerund 7 9 rain hissing
12539000998014861558 gerund 4 6 of exciting
12539000998014861558 gerund 6 8 serenely accepting
12539000998014861558 gerund 6 8 in accepting
12539000998014861558 gerund 1 3 and blinding
12539000998014861558 gerund 19 21 the paving
12539000998014861558 gerund 21 23 without noticing
12539000998014861558 gerund 15 17 , watching
125390009980

12539000998014861558 gerund 21 23 mother discussing
12539000998014861558 gerund 62 64 of failing
12539000998014861558 gerund 12 14 painfully struggling
12539000998014861558 gerund 3 5 off reading
12539000998014861558 gerund 21 23 , setting
12539000998014861558 gerund 29 31 seconds clouding
12539000998014861558 gerund 8 10 book waiting
12539000998014861558 gerund 35 37 , forcing
12539000998014861558 gerund 8 10 hillside blazing
12539000998014861558 gerund 17 19 trees shimmering
12539000998014861558 gerund 11 13 without being
12539000998014861558 gerund 5 7 by watching
12539000998014861558 gerund 10 12 without being
12539000998014861558 gerund 19 21 somehow avenging
12539000998014861558 gerund 5 7 in imagining
12539000998014861558 gerund 15 17 the following
12539000998014861558 gerund 13 15 , expecting
12539000998014861558 gerund 1 3 remember being
12539000998014861558 gerund 15 17 gone fishing
12539000998014861558 gerund 12 14 to learning
12539000998014861558 gerund 14 16 and trying
125

12539000998014861558 gerund 9 11 on maintaining
12539000998014861558 gerund 28 30 stopped playing
12539000998014861558 gerund 19 21 the following
12539000998014861558 gerund 2 4 stopped talking
12539000998014861558 gerund 12 14 before reading
tmp__en.txt
2037
tmp__ru.txt
2865
tmp__es.txt
1315
tmp__pt.txt
1467
tmp__it.txt
1291


In [13]:
#Gerund (w/o aux)

print('gerund w/o aux')
print('e.g. Without glancing')
#ru
oddsratio, pvalue = scipy.stats.fisher_exact([[2865, 156100], [3104, 155861]])
print('ru')
print(pvalue)

#pt
oddsratio, pvalue = scipy.stats.fisher_exact([[1467, 67592], [1349, 67710]])
print('pt')
print(pvalue)

#it
oddsratio, pvalue = scipy.stats.fisher_exact([[1291, 71626], [1424, 71493]])
print('it')
print(pvalue)

#es
oddsratio, pvalue = scipy.stats.fisher_exact([[1315, 70063], [1394, 69984]])
print('es')
print(pvalue)

gerund w/o aux
e.g. Without glancing
ru
0.0018694212168609233
pt
0.02588863225464761
it
0.010538242386526415
es
0.1302548567319941


#### 4.2.2 Gerund w/ aux

In [126]:
#aux + gerund/present particle
pattern = [{"DEP": "aux"}, {"TAG": "VBG"}]

matcher.add("gerund", None, pattern)
for filename in os.listdir(sourcefiledirectory):
    if filename.startswith("all"):
        
        f = open(filename, 'r')
        text = f.read()
        lines = sent_tokenize(text)
        outname = filename.replace('all', 'tmp_')
        with open(outname, 'w') as out:
            print(filename)
            for index,line in enumerate(lines):
                annotated_line = nlp(line)
                matches = matcher(annotated_line)
                for match_id, start, end in matches:
                    string_id = nlp.vocab.strings[match_id]  # Get string representation
                    span = annotated_line[start:end]  # The matched span
                    print(match_id, string_id, start, end, span.text)
                    out.write(str(match_id))
                    out.write("\n")
for filename in os.listdir(sourcefiledirectory):
    if filename.startswith("tmp_"):
        with open(filename, 'r') as f:
            outname = filename.replace('tmp_', 'AUXgerund_')
            with open(outname, 'w') as out:
                num_lines = 0
                for line in f:
                    num_lines += 1
                print(filename)
                print(num_lines)
                out.write(str(num_lines))
matcher.remove("gerund")

all_ru.txt
12539000998014861558 gerund 2 4 get going
12539000998014861558 gerund 29 31 was playing
12539000998014861558 gerund 10 12 was doing
12539000998014861558 gerund 1 3 were rolling
12539000998014861558 gerund 1 3 was hoping
12539000998014861558 gerund 26 28 was feeling
12539000998014861558 gerund 2 4 was reaping
12539000998014861558 gerund 8 10 to according
12539000998014861558 gerund 9 11 be raining
12539000998014861558 gerund 31 33 was dealing
12539000998014861558 gerund 10 12 was carrying
12539000998014861558 gerund 5 7 was being
12539000998014861558 gerund 2 4 was lying
12539000998014861558 gerund 5 7 been watching
12539000998014861558 gerund 6 8 was grabbing
12539000998014861558 gerund 1 3 was running
12539000998014861558 gerund 2 4 was choking
12539000998014861558 gerund 11 13 was writhing
12539000998014861558 gerund 12 14 was trying
12539000998014861558 gerund 20 22 was eating
12539000998014861558 gerund 18 20 was wearing
12539000998014861558 gerund 9 11 was holding
12539

12539000998014861558 gerund 10 12 are sleeping
12539000998014861558 gerund 1 3 is handing
12539000998014861558 gerund 12 14 is leaning
12539000998014861558 gerund 6 8 been counting
12539000998014861558 gerund 6 8 be working
12539000998014861558 gerund 24 26 was wearing
12539000998014861558 gerund 3 5 was thinking
12539000998014861558 gerund 2 4 were trembling
12539000998014861558 gerund 4 6 are giving
12539000998014861558 gerund 17 19 been feeling
12539000998014861558 gerund 11 13 be drinking
12539000998014861558 gerund 9 11 am going
12539000998014861558 gerund 10 12 be swimming
12539000998014861558 gerund 1 3 was sitting
12539000998014861558 gerund 11 13 was getting
12539000998014861558 gerund 1 3 was shaking
12539000998014861558 gerund 7 9 was standing
12539000998014861558 gerund 16 18 was faking
12539000998014861558 gerund 1 3 was sobbing
12539000998014861558 gerund 1 3 was staring
12539000998014861558 gerund 2 4 was leaving
12539000998014861558 gerund 2 4 be looking
125390009980148

12539000998014861558 gerund 25 27 was going
12539000998014861558 gerund 3 5 is going
12539000998014861558 gerund 33 35 been running
12539000998014861558 gerund 8 10 are bringing
12539000998014861558 gerund 6 8 been forgetting
12539000998014861558 gerund 1 3 was standing
12539000998014861558 gerund 12 14 was sitting
12539000998014861558 gerund 50 52 was lying
12539000998014861558 gerund 10 12 be lying
12539000998014861558 gerund 19 21 be edifying
12539000998014861558 gerund 33 35 was hanging
12539000998014861558 gerund 1 3 was wearing
12539000998014861558 gerund 11 13 been seeing
12539000998014861558 gerund 2 4 been counting
12539000998014861558 gerund 9 11 was living
12539000998014861558 gerund 16 18 been hoping
12539000998014861558 gerund 6 8 been counting
12539000998014861558 gerund 10 12 were drinking
12539000998014861558 gerund 5 7 was crossing
12539000998014861558 gerund 1 3 was leafing
12539000998014861558 gerund 10 12 was sitting
12539000998014861558 gerund 1 3 was running
12539

12539000998014861558 gerund 1 3 were talking
12539000998014861558 gerund 25 27 were talking
12539000998014861558 gerund 12 14 was bending
12539000998014861558 gerund 43 45 is disappearing
12539000998014861558 gerund 3 5 was hanging
12539000998014861558 gerund 5 7 is sitting
12539000998014861558 gerund 18 20 was being
12539000998014861558 gerund 32 34 am studying
12539000998014861558 gerund 42 44 is gathering
12539000998014861558 gerund 46 48 are swirling
12539000998014861558 gerund 5 7 was doing
12539000998014861558 gerund 8 10 are breaking
12539000998014861558 gerund 26 28 was being
12539000998014861558 gerund 14 16 was setting
12539000998014861558 gerund 30 32 was doing
12539000998014861558 gerund 83 85 be thinking
12539000998014861558 gerund 4 6 was thinking
12539000998014861558 gerund 32 34 been living
12539000998014861558 gerund 13 15 was delivering
12539000998014861558 gerund 68 70 was presenting
12539000998014861558 gerund 40 42 were trying
12539000998014861558 gerund 59 61 was 

12539000998014861558 gerund 39 41 was protecting
12539000998014861558 gerund 29 31 was loitering
12539000998014861558 gerund 1 3 is muttering
all_en.txt
12539000998014861558 gerund 11 13 been thinking
12539000998014861558 gerund 4 6 been thinking
12539000998014861558 gerund 20 22 been paying
12539000998014861558 gerund 4 6 was pulling
12539000998014861558 gerund 20 22 was reminding
12539000998014861558 gerund 9 11 was dying
12539000998014861558 gerund 20 22 was breaking
12539000998014861558 gerund 2 4 were coming
12539000998014861558 gerund 1 3 is building
12539000998014861558 gerund 13 15 are sending
12539000998014861558 gerund 11 13 been reporting
12539000998014861558 gerund 31 33 is falling
12539000998014861558 gerund 2 4 was sheeting
12539000998014861558 gerund 12 14 was racing
12539000998014861558 gerund 14 16 were laughing
12539000998014861558 gerund 31 33 was sleeping
12539000998014861558 gerund 33 35 were starting
12539000998014861558 gerund 2 4 was quivering
125390009980148615

...

12539000998014861558 gerund 26 28 were lacking
12539000998014861558 gerund 13 15 was holding
12539000998014861558 gerund 1 3 was walking
12539000998014861558 gerund 22 24 was wearing
12539000998014861558 gerund 13 15 was praying
12539000998014861558 gerund 7 9 was falling
12539000998014861558 gerund 12 14 was sailing
12539000998014861558 gerund 8 10 were eating
12539000998014861558 gerund 12 14 is doing
12539000998014861558 gerund 3 5 been drinking
12539000998014861558 gerund 30 32 were eating
12539000998014861558 gerund 4 6 be shielding
12539000998014861558 gerund 8 10 be screaming
12539000998014861558 gerund 33 35 been jumping
12539000998014861558 gerund 9 11 were crying
12539000998014861558 gerund 2 4 been expecting
12539000998014861558 gerund 47 49 were saying
12539000998014861558 gerund 40 42 remained waving
12539000998014861558 gerund 30 32 were answering
12539000998014861558 gerund 4 6 is throwing
12539000998014861558 gerund 21 23 were passing
12539000998014861558 gerund 7 9 was

12539000998014861558 gerund 17 19 are getting
12539000998014861558 gerund 36 38 is falling
12539000998014861558 gerund 6 8 are calling
12539000998014861558 gerund 7 9 were going
12539000998014861558 gerund 30 32 is interrogating
12539000998014861558 gerund 69 71 are heading
12539000998014861558 gerund 15 17 was rolling
12539000998014861558 gerund 78 80 was falling
12539000998014861558 gerund 6 8 was happening
12539000998014861558 gerund 16 18 be maintaining
12539000998014861558 gerund 116 118 were going
12539000998014861558 gerund 33 35 were speaking
12539000998014861558 gerund 11 13 were living
12539000998014861558 gerund 10 12 be listening
12539000998014861558 gerund 5 7 was taking
12539000998014861558 gerund 1 3 was sleeping
12539000998014861558 gerund 7 9 was going
12539000998014861558 gerund 4 6 was going
12539000998014861558 gerund 1 3 was watching
12539000998014861558 gerund 1 3 was dripping
12539000998014861558 gerund 11 13 been repeating
12539000998014861558 gerund 7 9 finishe

12539000998014861558 gerund 28 30 was seeing
12539000998014861558 gerund 13 15 was smashing
12539000998014861558 gerund 3 5 ’s trying
12539000998014861558 gerund 17 19 were praying
12539000998014861558 gerund 7 9 was staring
12539000998014861558 gerund 6 8 was watching
12539000998014861558 gerund 2 4 was trying
12539000998014861558 gerund 4 6 was spitting
12539000998014861558 gerund 2 4 is waving
12539000998014861558 gerund 12 14 is thinking
12539000998014861558 gerund 2 4 is wondering
12539000998014861558 gerund 14 16 are managing
12539000998014861558 gerund 11 13 been happening
12539000998014861558 gerund 8 10 was being
12539000998014861558 gerund 16 18 was happening
12539000998014861558 gerund 15 17 was consuming
12539000998014861558 gerund 13 15 is walking
12539000998014861558 gerund 6 8 is teaching
12539000998014861558 gerund 7 9 was looking
12539000998014861558 gerund 13 15 was going
12539000998014861558 gerund 3 5 was wearing
12539000998014861558 gerund 14 16 were returning
1253

12539000998014861558 gerund 4 6 was selling
12539000998014861558 gerund 11 13 were going
12539000998014861558 gerund 4 6 was tearing
12539000998014861558 gerund 8 10 be sleeping
12539000998014861558 gerund 9 11 were dimming
12539000998014861558 gerund 3 5 were seeing
12539000998014861558 gerund 16 18 were enjoying
12539000998014861558 gerund 9 11 was playing
12539000998014861558 gerund 23 25 was sunbathing
12539000998014861558 gerund 26 28 was growing
12539000998014861558 gerund 2 4 is glistening
12539000998014861558 gerund 4 6 was watching
12539000998014861558 gerund 4 6 be speaking
12539000998014861558 gerund 2 4 was putting
12539000998014861558 gerund 6 8 was lying
12539000998014861558 gerund 3 5 was going
12539000998014861558 gerund 20 22 were sleeping
12539000998014861558 gerund 7 9 was referring
12539000998014861558 gerund 1 3 was kidding
12539000998014861558 gerund 5 7 was sitting
12539000998014861558 gerund 21 23 was thickening
12539000998014861558 gerund 3 5 was dreaming
12539

12539000998014861558 gerund 1 3 're getting
12539000998014861558 gerund 2 4 'm going
12539000998014861558 gerund 27 29 are becoming
12539000998014861558 gerund 24 26 be occupying
12539000998014861558 gerund 32 34 was being
12539000998014861558 gerund 4 6 'm going
12539000998014861558 gerund 7 9 was gasping
12539000998014861558 gerund 33 35 were outlining
12539000998014861558 gerund 13 15 was splashing
12539000998014861558 gerund 1 3 was beating
12539000998014861558 gerund 3 5 was staring
12539000998014861558 gerund 25 27 was getting
12539000998014861558 gerund 1 3 was happening
12539000998014861558 gerund 13 15 was going
12539000998014861558 gerund 4 6 was going
12539000998014861558 gerund 30 32 was fixing
12539000998014861558 gerund 7 9 was taking
12539000998014861558 gerund 12 14 was combing
12539000998014861558 gerund 7 9 was sitting
12539000998014861558 gerund 1 3 was singing
12539000998014861558 gerund 1 3 was lying
12539000998014861558 gerund 4 6 was waiting
12539000998014861558 

In [ ]:
#Fisher's exact: gerund (w/ aux)

print('gerund w/ aux')
print('e.g. were rolling')

#ru
oddsratio, pvalue = scipy.stats.fisher_exact([[733, 78750], [660, 78823]])
print('ru')
print(pvalue)

#pt
oddsratio, pvalue = scipy.stats.fisher_exact([[274, 34256], [287, 34243]])
print('pt')
print(pvalue)

#it
oddsratio, pvalue = scipy.stats.fisher_exact([[250, 36209], [303, 36156]])
print('it')
print(pvalue)

#es
oddsratio, pvalue = scipy.stats.fisher_exact([[369, 35320], [296, 35393]])
print('es')
print(pvalue)

### 4.3 have
Russian expresses possession using a construction with the "be" verb. We wanted to see if this impacted the frequency with which "have" occurs (in the meaning of possession).

In [139]:
#have
pattern = [{}, {"DEP": "ROOT", "LEMMA": "have"}, {"LEMMA": {"NOT_IN": ["to"]}}]

matcher.add("have", None, pattern)
for filename in os.listdir(sourcefiledirectory):
    if filename.startswith("all"):
        
        f = open(filename, 'r')
        text = f.read()
        lines = sent_tokenize(text)
        outname = filename.replace('all', 'tmp_')
        with open(outname, 'w') as out:
            print(filename)
            for index,line in enumerate(lines):
                annotated_line = nlp(line)
                matches = matcher(annotated_line)
                for match_id, start, end in matches:
                    string_id = nlp.vocab.strings[match_id]  # Get string representation
                    span = annotated_line[start:end]  # The matched span
                    print(match_id, string_id, start, end, span.text)
                    out.write(str(match_id))
                    out.write("\n")
for filename in os.listdir(sourcefiledirectory):
    if filename.startswith("tmp_"):
        with open(filename, 'r') as f:
            outname = filename.replace('tmp_', 'have_')
            num_lines = 0
            with open(outname, 'w') as out:
                for line in f:
                    num_lines += 1
                print(filename)
                print(num_lines)
                out.write(str(num_lines))
matcher.remove("have")

all_ru.txt
14692702688101715474 have 6 9 n’t have enough
14692702688101715474 have 1 4 they hadn’t
14692702688101715474 have 2 5 n’t have too
14692702688101715474 have 3 6 inhabit have no
14692702688101715474 have 2 5 I had no
14692702688101715474 have 0 3 I had no
14692702688101715474 have 0 3 Farid had a
14692702688101715474 have 18 21 nearly had a
14692702688101715474 have 1 4 paramedic had no
14692702688101715474 have 0 3 I had no
14692702688101715474 have 2 5 you have for
14692702688101715474 have 1 4 I have a
14692702688101715474 have 15 18 n’t have any
14692702688101715474 have 10 13 I had no
14692702688101715474 have 1 4 bathroom had no
14692702688101715474 have 0 3 He had several
14692702688101715474 have 6 9 even have any
14692702688101715474 have 2 5 you have any
14692702688101715474 have 4 7 n’t have two
14692702688101715474 have 15 18 still hadn’t
14692702688101715474 have 3 6 even have a
14692702688101715474 have 6 9 would have plenty
14692702688101715474 have 15 18 you h

14692702688101715474 have 21 24 he had enough
14692702688101715474 have 2 5 ’ve had it
14692702688101715474 have 0 3 He had good
14692702688101715474 have 0 3 She had a
14692702688101715474 have 1 4 suddenly had the
14692702688101715474 have 1 4 I have a
14692702688101715474 have 1 4 half had its
14692702688101715474 have 0 3 I have a
14692702688101715474 have 15 18 girl has three
14692702688101715474 have 38 41 I have a
14692702688101715474 have 22 25 To have days
14692702688101715474 have 9 12 you have children
14692702688101715474 have 2 5 have had her
14692702688101715474 have 22 25 Yasha had a
14692702688101715474 have 4 7 you have us
14692702688101715474 have 3 6 I have you
14692702688101715474 have 3 6 n’t have me
14692702688101715474 have 0 3 He had a
14692702688101715474 have 1 4 already had the
14692702688101715474 have 1 4 always had difficulty
14692702688101715474 have 6 9 , had no
14692702688101715474 have 3 6 , have a
14692702688101715474 have 12 15 two have different
146

14692702688101715474 have 1 4 It has heating
14692702688101715474 have 8 11 scoreboard had us
14692702688101715474 have 30 33 have had a
14692702688101715474 have 8 11 had had the
14692702688101715474 have 4 7 n’t have a
14692702688101715474 have 0 3 Turner had warehouse
14692702688101715474 have 1 4 boy had stones
14692702688101715474 have 1 4 story had complicated
14692702688101715474 have 2 5 n’t had the
14692702688101715474 have 0 3 He had two
14692702688101715474 have 10 13 one had the
14692702688101715474 have 3 6 he had a
14692702688101715474 have 7 10 , had a
14692702688101715474 have 1 4 father had hold
14692702688101715474 have 0 3 She has a
14692702688101715474 have 0 3 Aidan has a
14692702688101715474 have 0 3 He has the
14692702688101715474 have 0 3 Declan has a
14692702688101715474 have 8 11 n’t have a
14692702688101715474 have 1 4 you have even
14692702688101715474 have 10 13 ’ll have one
14692702688101715474 have 9 12 n’t have one
14692702688101715474 have 1 4 ’ll have 

14692702688101715474 have 0 3 He had me
14692702688101715474 have 18 21 n’t have the
14692702688101715474 have 5 8 I had the
14692702688101715474 have 6 9 I had excerpts
14692702688101715474 have 1 4 even had a
14692702688101715474 have 0 3 I have a
14692702688101715474 have 18 21 , have a
14692702688101715474 have 11 14 I’ve

14692702688101715474 have 52 55 not have an
14692702688101715474 have 2 5 n’t have any
14692702688101715474 have 0 3 He has a
14692702688101715474 have 14 17 
haven’t
14692702688101715474 have 78 81 country has its
all_it.txt
14692702688101715474 have 1 4 he has the
14692702688101715474 have 0 3 He has a
14692702688101715474 have 0 3 He has a
14692702688101715474 have 0 3 He had metal
14692702688101715474 have 3 6 he had a
14692702688101715474 have 1 4 Spino had the
14692702688101715474 have 104 107 you have no
14692702688101715474 have 1 4 you have a
14692702688101715474 have 10 13 we have a
14692702688101715474 have 18 21 you had the
14692702688101715474 have 5

14692702688101715474 have 8 11 lady had a
14692702688101715474 have 25 28 you have a
14692702688101715474 have 0 3 She had no
14692702688101715474 have 3 6 bitches had it
14692702688101715474 have 5 8 Herrera had a
14692702688101715474 have 2 5 family had money
14692702688101715474 have 1 4 barely had time
14692702688101715474 have 0 3 We had no
14692702688101715474 have 0 3 I had trouble
14692702688101715474 have 7 10 they hadn’t
14692702688101715474 have 0 3 We had the
14692702688101715474 have 0 3 She had only
14692702688101715474 have 13 16 only had friends
14692702688101715474 have 0 3 Never had one
14692702688101715474 have 7 10 already had Paris
14692702688101715474 have 2 5 suddenly had an
14692702688101715474 have 8 11 n't have anyone
14692702688101715474 have 1 4 Each has his
14692702688101715474 have 3 6 nose has an
14692702688101715474 have 1 4 I've three
14692702688101715474 have 0 3 They had a
14692702688101715474 have 1 4 still have it
14692702688101715474 have 9 12 it h

In [140]:
#have

#ru
print('have (verb)')
oddsratio, pvalue = scipy.stats.fisher_exact([[300, 158665], [285, 158680]])
print('ru')#
print(pvalue)

#pt
oddsratio, pvalue = scipy.stats.fisher_exact([[91, 68968], [124, 68935]])
print('pt')
print(pvalue)

#it
oddsratio, pvalue = scipy.stats.fisher_exact([[106, 72811], [131, 72786]])
print('it')
print(pvalue)

#es
oddsratio, pvalue = scipy.stats.fisher_exact([[100, 71278], [128, 71250]])
print('es')
print(pvalue)

have (verb)
ru
0.5623819112020958
pt
0.02872162517390245
it
0.11851892736426234
es
0.07329512520471052


### 4.4 it's/that's
van Halteren (2008) satates that "it's" and "that's" are less frequent in translated texts in the Europarl corpus, so we wanted to see if we could find the same effect in these literary translations.

Problem: apostrophes are characters used when writing Python code. In theory they should be escapable... but none of my attempts to escape them succeeded. So I opened the corpora in a text editor and did a case-insensitive word count for "it's" and "that's" and added those numbers together, then put them in Excel for finding the values for Fisher's exact.


In [15]:
#that's/it's

#ru
print('that\'s/it\'s')
oddsratio, pvalue = scipy.stats.fisher_exact([[63, 158902], [20, 158945]])
print('ru')#
print(pvalue)

#pt
oddsratio, pvalue = scipy.stats.fisher_exact([[0, 69059], [9, 69050]])
print('pt')
print(pvalue)

#it
oddsratio, pvalue = scipy.stats.fisher_exact([[0, 72917], [9, 72908]])
print('it')
print(pvalue)

#es
oddsratio, pvalue = scipy.stats.fisher_exact([[14, 71364], [9, 71369]])
print('es')
print(pvalue)

that's/it's
ru
2.4236673299259728e-06
pt
0.0039052319187016407
it
0.0039052857821330624
es
0.4048346763407462


### 4.5 Past participles
Russian use of past participles differs from English, so we wanted to see if that had an impact on how past participles appear in the English translations.

In [90]:
#past participle
pattern = [{"DEP": "aux"}, {"TAG": "VBN"}, {}]

matcher.add("pastpart", None, pattern)
for filename in os.listdir(sourcefiledirectory):
    if filename.startswith("all"):
        
        f = open(filename, 'r')
        text = f.read()
        lines = sent_tokenize(text)
        outname = filename.replace('all', 'tmp_')
        with open(outname, 'w') as out:
            print(filename)
            for index,line in enumerate(lines):
                annotated_line = nlp(line)
                matches = matcher(annotated_line)
                for match_id, start, end in matches:
                    string_id = nlp.vocab.strings[match_id]  # Get string representation
                    span = annotated_line[start:end]  # The matched span
                    print(match_id, string_id, start, end, span.text)
                    out.write(str(match_id))
                    out.write("\n")
for filename in os.listdir(sourcefiledirectory):
    if filename.startswith("tmp_"):
        with open(filename, 'r') as f:
            outname = filename.replace('tmp_', 'pastpart_')
            num_lines = 0
            with open(outname, 'w') as out:
                for line in f:
                    num_lines += 1
                print(filename)
                print(num_lines)
                out.write(str(num_lines))
matcher.remove("pastpart")

all_ru.txt
4266497899917444067 pastpart 23 26 had pushed together
4266497899917444067 pastpart 10 13 had taken a
4266497899917444067 pastpart 2 5 had given way
4266497899917444067 pastpart 16 19 had gone out
4266497899917444067 pastpart 1 4 had lost part
4266497899917444067 pastpart 5 8 had been severely
4266497899917444067 pastpart 1 4 had received a
4266497899917444067 pastpart 36 39 had spelled out
4266497899917444067 pastpart 2 5 had tightened the
4266497899917444067 pastpart 14 17 have stopped the
4266497899917444067 pastpart 1 4 had dealt with
4266497899917444067 pastpart 1 4 had taken us
4266497899917444067 pastpart 1 4 had chosen this
4266497899917444067 pastpart 4 7 had been at
4266497899917444067 pastpart 18 21 had died.
4266497899917444067 pastpart 8 11 had become “
4266497899917444067 pastpart 20 23 had carried him
4266497899917444067 pastpart 14 17 have expected.
4266497899917444067 pastpart 4 7 had been watching
4266497899917444067 pastpart 1 4 had removed the
42664978999

4266497899917444067 pastpart 3 6 has been turned
4266497899917444067 pastpart 1 4 has matured in
4266497899917444067 pastpart 13 16 had dealt so
4266497899917444067 pastpart 20 23 has been illegally
4266497899917444067 pastpart 3 6 had been wrecked
4266497899917444067 pastpart 11 14 had awakened the
4266497899917444067 pastpart 11 14 had housed two
4266497899917444067 pastpart 3 6 had suffered for
4266497899917444067 pastpart 7 10 have paled at
4266497899917444067 pastpart 7 10 have been impossible
4266497899917444067 pastpart 27 30 had held the
4266497899917444067 pastpart 42 45 had come upon
4266497899917444067 pastpart 29 32 had stopped by
4266497899917444067 pastpart 2 5 have been in
4266497899917444067 pastpart 1 4 have been eaten
4266497899917444067 pastpart 11 14 had made nests
4266497899917444067 pastpart 18 21 had grown to
4266497899917444067 pastpart 1 4 had gained so
4266497899917444067 pastpart 9 12 had vanished—
4266497899917444067 pastpart 15 18 had turned it
426649789991

4266497899917444067 pastpart 34 37 have been just
4266497899917444067 pastpart 42 45 have been at
4266497899917444067 pastpart 4 7 had covered all
4266497899917444067 pastpart 16 19 have been a
4266497899917444067 pastpart 9 12 had gone missing
4266497899917444067 pastpart 10 13 had arranged for
4266497899917444067 pastpart 31 34 had been buried
4266497899917444067 pastpart 8 11 had gotten along
4266497899917444067 pastpart 28 31 had dressed and
4266497899917444067 pastpart 40 43 had forbidden her
4266497899917444067 pastpart 21 24 had seen a
4266497899917444067 pastpart 36 39 had changed.
4266497899917444067 pastpart 10 13 had put Tamara
4266497899917444067 pastpart 45 48 had been finished
4266497899917444067 pastpart 51 54 have dissolved,
4266497899917444067 pastpart 63 66 have traveled all
4266497899917444067 pastpart 5 8 having decided he
4266497899917444067 pastpart 3 6 had offered several
4266497899917444067 pastpart 3 6 had grown fond
4266497899917444067 pastpart 9 12 had begun.

4266497899917444067 pastpart 8 11 had gotten used
4266497899917444067 pastpart 2 5 had been prompted
4266497899917444067 pastpart 4 7 had frightened him
4266497899917444067 pastpart 11 14 had been in
4266497899917444067 pastpart 1 4 had been nicknamed
4266497899917444067 pastpart 9 12 had grown to
4266497899917444067 pastpart 2 5 have divorced,
4266497899917444067 pastpart 11 14 'd frightened her
4266497899917444067 pastpart 18 21 had earned his
4266497899917444067 pastpart 12 15 had striven for
4266497899917444067 pastpart 26 29 had been ready
4266497899917444067 pastpart 3 6 had devoured her
4266497899917444067 pastpart 4 7 have cared less
4266497899917444067 pastpart 8 11 had hit whom
4266497899917444067 pastpart 32 35 have shared it
4266497899917444067 pastpart 2 5 have tarnished her
4266497899917444067 pastpart 64 67 had ended up
4266497899917444067 pastpart 3 6 have kicked the
4266497899917444067 pastpart 23 26 had grown tired
4266497899917444067 pastpart 3 6 had been an
42664978

4266497899917444067 pastpart 36 39 had built in
4266497899917444067 pastpart 4 7 had rented out
4266497899917444067 pastpart 29 32 had gone for
4266497899917444067 pastpart 2 5 had been taken
4266497899917444067 pastpart 6 9 had reached draft
4266497899917444067 pastpart 2 5 had seen their
4266497899917444067 pastpart 2 5 had brought firewood
4266497899917444067 pastpart 16 19 have slept poorly
4266497899917444067 pastpart 4 7 had stashed away
4266497899917444067 pastpart 9 12 has come into
4266497899917444067 pastpart 5 8 have seen a
4266497899917444067 pastpart 5 8 had gone to
4266497899917444067 pastpart 10 13 had said then
4266497899917444067 pastpart 2 5 have called you
4266497899917444067 pastpart 10 13 had started to
4266497899917444067 pastpart 4 7 had passed since
4266497899917444067 pastpart 1 4 had happened,
4266497899917444067 pastpart 1 4 have left the
4266497899917444067 pastpart 8 11 have moved in
4266497899917444067 pastpart 2 5 had married recently
4266497899917444067 

...

4266497899917444067 pastpart 29 32 do left to
4266497899917444067 pastpart 4 7 had been wandering
4266497899917444067 pastpart 22 25 had introduced themselves
4266497899917444067 pastpart 3 6 had reinforced the
4266497899917444067 pastpart 31 34 had been returned
4266497899917444067 pastpart 37 40 had acquired on
4266497899917444067 pastpart 3 6 had been enough
4266497899917444067 pastpart 38 41 had been performed
4266497899917444067 pastpart 28 31 had resumed,
4266497899917444067 pastpart 50 53 had duelled to
4266497899917444067 pastpart 5 8 had been sprayed
4266497899917444067 pastpart 9 12 had fought vigorously
4266497899917444067 pastpart 1 4 had challenged,
4266497899917444067 pastpart 2 5 had decided.
4266497899917444067 pastpart 21 24 had joined the
4266497899917444067 pastpart 1 4 had been transported
4266497899917444067 pastpart 4 7 had been given
4266497899917444067 pastpart 15 18 had been dismissed
4266497899917444067 pastpart 19 22 had been present
4266497899917444067 pastp

4266497899917444067 pastpart 9 12 had posted a
4266497899917444067 pastpart 19 22 had made reprisals
4266497899917444067 pastpart 56 59 had left with
4266497899917444067 pastpart 4 7 had arrived Wednesday
4266497899917444067 pastpart 24 27 had left some
4266497899917444067 pastpart 36 39 had killed the
4266497899917444067 pastpart 21 24 has sprinkled it
4266497899917444067 pastpart 2 5 has been no
4266497899917444067 pastpart 1 4 had been captured
4266497899917444067 pastpart 8 11 had impressed even
4266497899917444067 pastpart 26 29 had gone so
4266497899917444067 pastpart 3 6 had informed me
4266497899917444067 pastpart 30 33 have done no
4266497899917444067 pastpart 18 21 had been up
4266497899917444067 pastpart 11 14 had asked,
4266497899917444067 pastpart 15 18 had tortured me
4266497899917444067 pastpart 24 27 have uttered a
4266497899917444067 pastpart 13 16 had laid down
4266497899917444067 pastpart 14 17 had happened in
4266497899917444067 pastpart 7 10 have felt a
42664978999

4266497899917444067 pastpart 8 11 had been recognized
4266497899917444067 pastpart 6 9 had known that
4266497899917444067 pastpart 18 21 had remained intact
4266497899917444067 pastpart 5 8 have prevented her
4266497899917444067 pastpart 1 4 had resorted to
4266497899917444067 pastpart 15 18 had been obliged
4266497899917444067 pastpart 14 17 had done a
4266497899917444067 pastpart 43 46 having collected themselves
4266497899917444067 pastpart 11 14 felt obliged to
4266497899917444067 pastpart 24 27 have been related
4266497899917444067 pastpart 5 8 had curled into
4266497899917444067 pastpart 13 16 have had fatal
4266497899917444067 pastpart 3 6 has forgotten what
4266497899917444067 pastpart 2 5 have made love
4266497899917444067 pastpart 18 21 has become of
4266497899917444067 pastpart 8 11 have seemed like
4266497899917444067 pastpart 54 57 have got splashed
4266497899917444067 pastpart 69 72 had come to
4266497899917444067 pastpart 20 23 have been a
4266497899917444067 pastpart 17

4266497899917444067 pastpart 35 38 had made it
4266497899917444067 pastpart 13 16 had become.
4266497899917444067 pastpart 38 41 had handled things
4266497899917444067 pastpart 3 6 had been assigned
4266497899917444067 pastpart 3 6 have come from
4266497899917444067 pastpart 23 26 had left lying
4266497899917444067 pastpart 21 24 have thought of
4266497899917444067 pastpart 12 15 had arrived.
4266497899917444067 pastpart 4 7 have been uncomfortable
4266497899917444067 pastpart 7 10 have gone down
4266497899917444067 pastpart 13 16 had taken out
4266497899917444067 pastpart 12 15 had been.
4266497899917444067 pastpart 1 4 had kicked over
4266497899917444067 pastpart 9 12 had remembered to
4266497899917444067 pastpart 5 8 had fallen off
4266497899917444067 pastpart 2 5 had come.
4266497899917444067 pastpart 9 12 had gone to
4266497899917444067 pastpart 11 14 had turned to
4266497899917444067 pastpart 47 50 have given way
4266497899917444067 pastpart 3 6 had stopped and
426649789991744406

4266497899917444067 pastpart 2 5 had finished,
4266497899917444067 pastpart 12 15 had found her
4266497899917444067 pastpart 36 39 'd reported her
4266497899917444067 pastpart 31 34 has run its
4266497899917444067 pastpart 53 56 have been feeling
4266497899917444067 pastpart 25 28 've got it
4266497899917444067 pastpart 21 24 've given up
4266497899917444067 pastpart 4 7 had sealed a
4266497899917444067 pastpart 42 45 've found my
4266497899917444067 pastpart 18 21 had led me
4266497899917444067 pastpart 38 41 had placed their
4266497899917444067 pastpart 9 12 have guided the
4266497899917444067 pastpart 1 4 had been in
4266497899917444067 pastpart 1 4 had gone back
4266497899917444067 pastpart 28 31 had sapped him
4266497899917444067 pastpart 5 8 having aged prematurely
4266497899917444067 pastpart 4 7 have been blowing
4266497899917444067 pastpart 29 32 had gone crazy
4266497899917444067 pastpart 1 4 had resolved to
4266497899917444067 pastpart 29 32 had twisted my
426649789991744406

In [101]:
#Past participle
#ru
print('past participle')
print('e.g. had been wheeling, had killed')
oddsratio, pvalue = scipy.stats.fisher_exact([[1071, 78412], [861, 78621]])
print('ru')#
print(pvalue)

#pt
oddsratio, pvalue = scipy.stats.fisher_exact([[574, 33956], [374, 34155]])
print('pt')
print(pvalue)

#it
oddsratio, pvalue = scipy.stats.fisher_exact([[505, 35954], [395, 36064]])
print('it')
print(pvalue)

#es
oddsratio, pvalue = scipy.stats.fisher_exact([[510, 35179], [387, 35302]])
print('es')
print(pvalue)

past participle
e.g. had been wheeling, had killed
ru
1.6803138307595173e-06
pt
6.514445746234154e-11
it
0.0002521457251525573
es
4.039500439603572e-05


### 4.6 Some
We thought there might be some difference in the frequency of "some" in English for translations of Russian, because there are multiple words that could be translated that way.

In [106]:
#"some"
pattern = [{"POS": "DET", "LOWER": "some"}]

matcher.add("SOME", None, pattern)
for filename in os.listdir(sourcefiledirectory):
    if filename.startswith("all"):
        
        f = open(filename, 'r')
        text = f.read()
        lines = sent_tokenize(text)
        outname = filename.replace('all', 'tmp_')
        with open(outname, 'w') as out:
            print(filename)
            for index,line in enumerate(lines):
                annotated_line = nlp(line)
                matches = matcher(annotated_line)
                for match_id, start, end in matches:
                    string_id = nlp.vocab.strings[match_id]  # Get string representation
                    span = annotated_line[start:end]  # The matched span
                    print(match_id, string_id, start, end, span.text)
                    out.write(str(match_id))
                    out.write("\n")
for filename in os.listdir(sourcefiledirectory):
    if filename.startswith("tmp_"):
        with open(filename, 'r') as f:
            outname = filename.replace('tmp_', 'DET_some')
            num_lines = 0
            with open(outname, 'w') as out:
                for line in f:
                    num_lines += 1
                print(filename)
                print(num_lines)
                out.write(str(num_lines))
matcher.remove("SOME")

all_ru.txt
15016112045944751253 SOME 20 21 some
15016112045944751253 SOME 14 15 some
15016112045944751253 SOME 33 34 some
15016112045944751253 SOME 12 13 some
15016112045944751253 SOME 28 29 some
15016112045944751253 SOME 17 18 some
15016112045944751253 SOME 0 1 Some
15016112045944751253 SOME 17 18 some
15016112045944751253 SOME 5 6 some
15016112045944751253 SOME 7 8 some
15016112045944751253 SOME 19 20 some
15016112045944751253 SOME 4 5 some
15016112045944751253 SOME 8 9 some
15016112045944751253 SOME 10 11 some
15016112045944751253 SOME 1 2 some
15016112045944751253 SOME 11 12 some
15016112045944751253 SOME 5 6 some
15016112045944751253 SOME 12 13 some
15016112045944751253 SOME 18 19 some
15016112045944751253 SOME 16 17 some
15016112045944751253 SOME 5 6 some
15016112045944751253 SOME 0 1 Some
15016112045944751253 SOME 12 13 some
15016112045944751253 SOME 35 36 some
15016112045944751253 SOME 4 5 some
15016112045944751253 SOME 2 3 some
15016112045944751253 SOME 4 5 some
15016112045944

15016112045944751253 SOME 112 113 some
15016112045944751253 SOME 27 28 some
15016112045944751253 SOME 4 5 some
15016112045944751253 SOME 0 1 Some
15016112045944751253 SOME 22 23 some
15016112045944751253 SOME 13 14 some
15016112045944751253 SOME 17 18 some
15016112045944751253 SOME 25 26 some
15016112045944751253 SOME 5 6 some
15016112045944751253 SOME 11 12 some
15016112045944751253 SOME 9 10 some
15016112045944751253 SOME 4 5 some
15016112045944751253 SOME 42 43 some
15016112045944751253 SOME 8 9 some
15016112045944751253 SOME 46 47 some
15016112045944751253 SOME 1 2 some
15016112045944751253 SOME 14 15 some
15016112045944751253 SOME 1 2 Some
15016112045944751253 SOME 0 1 Some
15016112045944751253 SOME 9 10 some
15016112045944751253 SOME 26 27 some
15016112045944751253 SOME 4 5 some
15016112045944751253 SOME 19 20 some
15016112045944751253 SOME 2 3 some
15016112045944751253 SOME 8 9 some
15016112045944751253 SOME 16 17 some
15016112045944751253 SOME 24 25 some
15016112045944751253 SO

15016112045944751253 SOME 25 26 some
15016112045944751253 SOME 14 15 some
15016112045944751253 SOME 9 10 some
15016112045944751253 SOME 8 9 some
15016112045944751253 SOME 5 6 some
15016112045944751253 SOME 0 1 Some
15016112045944751253 SOME 7 8 some
15016112045944751253 SOME 4 5 some
15016112045944751253 SOME 34 35 some
15016112045944751253 SOME 12 13 some
15016112045944751253 SOME 24 25 some
15016112045944751253 SOME 10 11 some
15016112045944751253 SOME 8 9 some
15016112045944751253 SOME 0 1 Some
15016112045944751253 SOME 7 8 some
15016112045944751253 SOME 0 1 Some
15016112045944751253 SOME 25 26 some
15016112045944751253 SOME 0 1 Some
15016112045944751253 SOME 23 24 some
15016112045944751253 SOME 11 12 some
15016112045944751253 SOME 32 33 some
15016112045944751253 SOME 14 15 some
15016112045944751253 SOME 8 9 some
15016112045944751253 SOME 33 34 some
15016112045944751253 SOME 52 53 some
15016112045944751253 SOME 41 42 some
15016112045944751253 SOME 12 13 some
15016112045944751253 SOM

15016112045944751253 SOME 11 12 some
15016112045944751253 SOME 1 2 some
15016112045944751253 SOME 8 9 some
15016112045944751253 SOME 83 84 some
15016112045944751253 SOME 5 6 some
15016112045944751253 SOME 9 10 some
15016112045944751253 SOME 4 5 some
15016112045944751253 SOME 9 10 some
15016112045944751253 SOME 0 1 Some
15016112045944751253 SOME 3 4 some
15016112045944751253 SOME 51 52 some
15016112045944751253 SOME 0 1 Some
15016112045944751253 SOME 22 23 some
15016112045944751253 SOME 7 8 some
15016112045944751253 SOME 23 24 some
15016112045944751253 SOME 19 20 some
15016112045944751253 SOME 0 1 Some
15016112045944751253 SOME 7 8 some
15016112045944751253 SOME 17 18 some
15016112045944751253 SOME 13 14 some
15016112045944751253 SOME 12 13 some
15016112045944751253 SOME 3 4 some
15016112045944751253 SOME 34 35 some
15016112045944751253 SOME 2 3 some
15016112045944751253 SOME 7 8 some
15016112045944751253 SOME 6 7 some
15016112045944751253 SOME 12 13 some
15016112045944751253 SOME 17 18

In [107]:
#some
print('some')

#ru
oddsratio, pvalue = scipy.stats.fisher_exact([[258, 158707], [209, 158756]])
print('ru')
print(pvalue)

#pt
oddsratio, pvalue = scipy.stats.fisher_exact([[113, 68946], [91, 68968]])
print('pt')
print(pvalue)

#it
oddsratio, pvalue = scipy.stats.fisher_exact([[117, 72800], [96, 72821]])
print('it')
print(pvalue)

#es
oddsratio, pvalue = scipy.stats.fisher_exact([[119, 71259], [94, 71284]])
print('es')
print(pvalue)

some
ru
0.02612342595981299
pt
0.14099968857951017
it
0.17011048442477092
es
0.09960340407369353


### 4.7 'that'
Olohan (2001) states that translated texts have a higher frequency of "that" as a complementizer after reporting verbs (e.g. 'said that'). We looked at it (as *'that'-ADP* for the part-of-speech marker used in Spacy for that construction), as well as the use of "that" as a determiner.

#### 4.7.1 'That' ADP

In [108]:
#ThatADP - with verb beforehand

pattern = [{"POS": "VERB"},{"LEMMA": "that", "POS": "ADP"},{}]
matcher.add("thatADP", None, pattern)
for filename in os.listdir(sourcefiledirectory):
    if filename.startswith("all"):
        
        f = open(filename, 'r')
        text = f.read()
    
        
        
        
        lines = sent_tokenize(text)
        outname = filename.replace('all', 'tmp_')
        with open(outname, 'w') as out:
            for index,line in enumerate(lines):
                annotated_line = nlp(line)
                matches = matcher(annotated_line)
                for match_id, start, end in matches:
                    string_id = nlp.vocab.strings[match_id]  # Get string representation
                    span = annotated_line[start:end]  # The matched span
                    print(match_id, string_id, start, end, span.text)
                    out.write(str(match_id))
                    out.write("\n")
for filename in os.listdir(sourcefiledirectory):
    if filename.startswith("tmp_"):
        with open(filename, 'r') as f:
            outname = filename.replace('tmp_', 'thatVERB_ADP_')
            num_lines = 0
            with open(outname, 'w') as out:
                for line in f:
                    num_lines += 1
                print(num_lines)
                out.write(str(num_lines))
matcher.remove("thatADP")

17860752185314450104 thatADP 3 6 say that in
17860752185314450104 thatADP 2 5 hoping that today
17860752185314450104 thatADP 1 4 say that after
17860752185314450104 thatADP 17 20 praying that the
17860752185314450104 thatADP 2 5 imagined that the
17860752185314450104 thatADP 20 23 saying that it
17860752185314450104 thatADP 11 14 said that a
17860752185314450104 thatADP 3 6 say that week
17860752185314450104 thatADP 24 27 noticed that instead
17860752185314450104 thatADP 1 4 said that it
17860752185314450104 thatADP 11 14 noticed that the
17860752185314450104 thatADP 4 7 suggested that the
17860752185314450104 thatADP 3 6 say that at
17860752185314450104 thatADP 1 4 realized that the
17860752185314450104 thatADP 4 7 noticed that the
17860752185314450104 thatADP 3 6 know that daiquiris
17860752185314450104 thatADP 2 5 heard that before
17860752185314450104 thatADP 3 6 reminded that the
17860752185314450104 thatADP 1 4 said that this
17860752185314450104 thatADP 8 11 say that nothing
178

17860752185314450104 thatADP 9 12 say that such
17860752185314450104 thatADP 1 4 felt that Russians
17860752185314450104 thatADP 1 4 said that city
17860752185314450104 thatADP 3 6 thought that when
17860752185314450104 thatADP 5 8 realize that the
17860752185314450104 thatADP 2 5 realized that it
17860752185314450104 thatADP 10 13 is that you
17860752185314450104 thatADP 8 11 realized that its
17860752185314450104 thatADP 70 73 seemed that at
17860752185314450104 thatADP 28 31 decided that the
17860752185314450104 thatADP 2 5 noticed that the
17860752185314450104 thatADP 14 17 suppose that they
17860752185314450104 thatADP 2 5 see that one
17860752185314450104 thatADP 8 11 saw that tears
17860752185314450104 thatADP 36 39 thought that both
17860752185314450104 thatADP 1 4 thought that this
17860752185314450104 thatADP 1 4 suspected that Grandmother
17860752185314450104 thatADP 37 40 thought that if
17860752185314450104 thatADP 105 108 announced that she
17860752185314450104 thatADP 1 

17860752185314450104 thatADP 2 5 grant that I
17860752185314450104 thatADP 10 13 grant that I
17860752185314450104 thatADP 4 7 doubted that it
17860752185314450104 thatADP 1 4 marveled that he
17860752185314450104 thatADP 37 40 think that a
17860752185314450104 thatADP 3 6 is that later
17860752185314450104 thatADP 5 8 know that Pushkin
17860752185314450104 thatADP 1 4 means that there
17860752185314450104 thatADP 15 18 planned that he
17860752185314450104 thatADP 1 4 knew that it
17860752185314450104 thatADP 11 14 suffered that much
17860752185314450104 thatADP 2 5 felt that he
17860752185314450104 thatADP 20 23 suggested that the
17860752185314450104 thatADP 2 5 reasoned that if
17860752185314450104 thatADP 3 6 made that crashing
17860752185314450104 thatADP 6 9 confirm that she
17860752185314450104 thatADP 10 13 learn that she
17860752185314450104 thatADP 2 5 wrote that she
17860752185314450104 thatADP 30 33 learn that My
17860752185314450104 thatADP 8 11 realize that she
1786075218

17860752185314450104 thatADP 1 4 lookit that fiiiine
17860752185314450104 thatADP 2 5 say that now
17860752185314450104 thatADP 11 14 mention that the
17860752185314450104 thatADP 2 5 think that highly
17860752185314450104 thatADP 2 5 reckon that,
17860752185314450104 thatADP 4 7 say that he
17860752185314450104 thatADP 6 9 knew that the
17860752185314450104 thatADP 21 24 felt that “
17860752185314450104 thatADP 1 4 knew that he
17860752185314450104 thatADP 2 5 happened that his
17860752185314450104 thatADP 14 17 proposed that a
17860752185314450104 thatADP 5 8 thought that general
17860752185314450104 thatADP 32 35 ordering that they
17860752185314450104 thatADP 4 7 announced that the
17860752185314450104 thatADP 37 40 notice that the
17860752185314450104 thatADP 18 21 learn that he
17860752185314450104 thatADP 1 4 learned that the
17860752185314450104 thatADP 46 49 sensed that his
17860752185314450104 thatADP 12 15 affirmed that not
17860752185314450104 thatADP 7 10 realized that she

17860752185314450104 thatADP 1 4 see that I
17860752185314450104 thatADP 3 6 thinks that he
17860752185314450104 thatADP 2 5 realize that if
17860752185314450104 thatADP 10 13 meets that of
17860752185314450104 thatADP 1 4 answered that maybe
17860752185314450104 thatADP 3 6 explained that he
17860752185314450104 thatADP 4 7 thinking that the
17860752185314450104 thatADP 8 11 answered that he
17860752185314450104 thatADP 36 39 realize that there
17860752185314450104 thatADP 68 71 proposed that the
17860752185314450104 thatADP 6 9 realize that these
17860752185314450104 thatADP 45 48 considering that the
17860752185314450104 thatADP 32 35 notice that the
17860752185314450104 thatADP 2 5 believe that you
17860752185314450104 thatADP 15 18 indicating that you
17860752185314450104 thatADP 39 42 ended— that I
17860752185314450104 thatADP 10 13 say that the
17860752185314450104 thatADP 23 26 recalled that for
17860752185314450104 thatADP 10 13 realized that there
17860752185314450104 thatADP

17860752185314450104 thatADP 18 21 believe that the
17860752185314450104 thatADP 2 5 proves that our
17860752185314450104 thatADP 3 6 felt that your
17860752185314450104 thatADP 3 6 remember that that
17860752185314450104 thatADP 6 9 sees that line
17860752185314450104 thatADP 1 4 notices that they
17860752185314450104 thatADP 3 6 realize that his
17860752185314450104 thatADP 1 4 realizes that the
17860752185314450104 thatADP 24 27 creams that none
17860752185314450104 thatADP 1 4 prays that a
17860752185314450104 thatADP 7 10 noticed that she
17860752185314450104 thatADP 59 62 muck that people
17860752185314450104 thatADP 19 22 feels that his
17860752185314450104 thatADP 2 5 knows that it
17860752185314450104 thatADP 1 4 knows that,
17860752185314450104 thatADP 1 4 realizes that this
17860752185314450104 thatADP 1 4 knows that the
17860752185314450104 thatADP 4 7 knowing that it
17860752185314450104 thatADP 5 8 bet that for
17860752185314450104 thatADP 148 151 given that the
178607521

17860752185314450104 thatADP 29 32 said that the
17860752185314450104 thatADP 11 14 said that it
17860752185314450104 thatADP 5 8 thought that I
17860752185314450104 thatADP 12 15 think that maybe
17860752185314450104 thatADP 3 6 realized that this
17860752185314450104 thatADP 27 30 shouted that he
17860752185314450104 thatADP 6 9 saw that he
17860752185314450104 thatADP 23 26 murmured that it
17860752185314450104 thatADP 1 4 admitted that the
17860752185314450104 thatADP 13 16 perceive that she
17860752185314450104 thatADP 34 37 declared that they
17860752185314450104 thatADP 9 12 knew that before
17860752185314450104 thatADP 10 13 was that,
17860752185314450104 thatADP 10 13 realized that it
17860752185314450104 thatADP 52 55 knew that she
17860752185314450104 thatADP 40 43 understood that the
17860752185314450104 thatADP 14 17 knew that she
17860752185314450104 thatADP 6 9 perceived that Dulce
17860752185314450104 thatADP 23 26 realized that she
17860752185314450104 thatADP 1 4 knew

#### Fisher's Exact or verb + 'that' ADP

In [109]:
#verb + thatADP

print('verb + that (ADP)')
print('e.g. said that, hoped that')

#ru
oddsratio, pvalue = scipy.stats.fisher_exact([[339, 79144], [241, 79242]])
print('ru')
print(pvalue)

#pt
oddsratio, pvalue = scipy.stats.fisher_exact([[161, 34369], [105, 34425]])
print('pt')
print(pvalue)

#it
oddsratio, pvalue = scipy.stats.fisher_exact([[202, 36257], [110, 36348]])
print('it')
print(pvalue)

#es
oddsratio, pvalue = scipy.stats.fisher_exact([[234, 35455], [108, 35581]])
print('es')
print(pvalue)

verb + that (ADP)
e.g. said that, hoped that
ru
5.255615284219157e-05
pt
0.000699201959593659
it
2.0023194087936434e-07
es
7.246397139358926e-12


#### 4.7.2 That (determiner)

In [111]:
#ThatDET - real one, with DT tag plus POS det
pattern = [{}, {"LEMMA": "that", "POS": "DET", "TAG": "DT"}, {}]
matcher.add("thatDET", None, pattern)
for filename in os.listdir(sourcefiledirectory):
    if filename.startswith("all"):
        
        f = open(filename, 'r')
        text = f.read()
        lines = sent_tokenize(text)
        outname = filename.replace('all', 'tmp')
        with open(outname, 'w') as out:
            for index,line in enumerate(lines):
                annotated_line = nlp(line)
                matches = matcher(annotated_line)
                for match_id, start, end in matches:
                    string_id = nlp.vocab.strings[match_id]  # Get string representation
                    span = annotated_line[start:end]  # The matched span
                    print(match_id, string_id, start, end, span.text)
                    out.write(str(match_id))
                    out.write("\n")
for filename in os.listdir(sourcefiledirectory):
    if filename.startswith("tmp_"):
        with open(filename, 'r') as f:
            outname = filename.replace('tmp', 'thatDET2')
            num_lines = 0
            with open(outname, 'w') as out:
                for line in f:
                    num_lines += 1
                print(num_lines)
                out.write(str(num_lines))
matcher.remove("thatDET")

1013076350897071439 thatDET 14 17 understood that was
1013076350897071439 thatDET 0 3 After that everyone
1013076350897071439 thatDET 3 6 remember that was
1013076350897071439 thatDET 15 18 like that,
1013076350897071439 thatDET 20 23 certainty that homicide
1013076350897071439 thatDET 3 6 do that.
1013076350897071439 thatDET 11 14 

That was
1013076350897071439 thatDET 9 12 like that anyway
1013076350897071439 thatDET 0 3 Not that sort
1013076350897071439 thatDET 8 11 at that point
1013076350897071439 thatDET 5 8 be that same
1013076350897071439 thatDET 26 29 backgammon that night
1013076350897071439 thatDET 2 5 was that?
1013076350897071439 thatDET 0 3 On that day
1013076350897071439 thatDET 5 8 take that one
1013076350897071439 thatDET 0 3 Put that prizefighter
1013076350897071439 thatDET 2 5 in that empty
1013076350897071439 thatDET 4 7 of that little
1013076350897071439 thatDET 1 4 only that,
1013076350897071439 thatDET 0 3 After that he
1013076350897071439 thatDET 20 23 time that

1013076350897071439 thatDET 4 7 like that,
1013076350897071439 thatDET 7 10 in that stuffy
1013076350897071439 thatDET 3 6 say that?
1013076350897071439 thatDET 20 23 over that which
1013076350897071439 thatDET 14 17 , that double
1013076350897071439 thatDET 1 4 remembered that day
1013076350897071439 thatDET 3 6 of that bum
1013076350897071439 thatDET 26 29 And that’s
1013076350897071439 thatDET 0 3 After that,
1013076350897071439 thatDET 3 6 of that one
1013076350897071439 thatDET 10 13 on that day
1013076350897071439 thatDET 1 4 His that!
1013076350897071439 thatDET 0 3 “That’s
1013076350897071439 thatDET 14 17 about that student
1013076350897071439 thatDET 2 5 beforehand that just
1013076350897071439 thatDET 9 12 of that name
1013076350897071439 thatDET 25 28 me that demagoguery
1013076350897071439 thatDET 0 3 So that was
1013076350897071439 thatDET 9 12 —that’s
1013076350897071439 thatDET 0 3 Earlier that day
1013076350897071439 thatDET 6 9 even that.
1013076350897071439 thatDET 1

1013076350897071439 thatDET 13 16 after that we
1013076350897071439 thatDET 12 15 from that memorable
1013076350897071439 thatDET 17 20 and that altered
1013076350897071439 thatDET 2 5 put that clock
1013076350897071439 thatDET 44 47 until that moment
1013076350897071439 thatDET 84 87 of that house
1013076350897071439 thatDET 8 11 

That’s
1013076350897071439 thatDET 3 6 easier that way
1013076350897071439 thatDET 0 3 On that Sunday
1013076350897071439 thatDET 3 6 herself that question
1013076350897071439 thatDET 0 3 At that time
1013076350897071439 thatDET 19 22 of that Rusalka
1013076350897071439 thatDET 4 7 hear that admission
1013076350897071439 thatDET 16 19 of that marvelous
1013076350897071439 thatDET 7 10 of that idea
1013076350897071439 thatDET 7 10 on that side
1013076350897071439 thatDET 17 20 , that very
1013076350897071439 thatDET 1 4 Since that day
1013076350897071439 thatDET 4 7 have that kind
1013076350897071439 thatDET 20 23 For that,
1013076350897071439 thatDET 27 30 

1013076350897071439 thatDET 4 7 , that's
1013076350897071439 thatDET 5 8 in that time
1013076350897071439 thatDET 1 4 from that moment
1013076350897071439 thatDET 6 9 and that was
1013076350897071439 thatDET 23 26 up that one
1013076350897071439 thatDET 7 10 hke that of
1013076350897071439 thatDET 6 9 when that nice
1013076350897071439 thatDET 2 5 of that you
1013076350897071439 thatDET 13 16 —that is
1013076350897071439 thatDET 1 4 in that capacity
1013076350897071439 thatDET 11 14 write that.
1013076350897071439 thatDET 1 4 n't that right
1013076350897071439 thatDET 4 7 wrote that,
1013076350897071439 thatDET 10 13 like that:
1013076350897071439 thatDET 14 17 like that.
1013076350897071439 thatDET 14 17 from that failure
1013076350897071439 thatDET 0 3 Igorseized that hand
1013076350897071439 thatDET 3 6 of that circle
1013076350897071439 thatDET 3 6 of that which
1013076350897071439 thatDET 12 15 in that same
1013076350897071439 thatDET 2 5 Oblachkin!^ That was
1013076350897071439 t

1013076350897071439 thatDET 5 8 of that?
1013076350897071439 thatDET 7 10 , that seemed
1013076350897071439 thatDET 5 8 of that than
1013076350897071439 thatDET 0 3 After that,
1013076350897071439 thatDET 23 26 and that was
1013076350897071439 thatDET 1 4 dinner that night
1013076350897071439 thatDET 2 5 ’s that?
1013076350897071439 thatDET 7 10 there that month
1013076350897071439 thatDET 4 7 at that while
1013076350897071439 thatDET 15 18 had that skill
1013076350897071439 thatDET 12 15 but that was
1013076350897071439 thatDET 13 16 like that before
1013076350897071439 thatDET 2 5 , that’s
1013076350897071439 thatDET 39 42 were that had
1013076350897071439 thatDET 0 3 Earlier that day
1013076350897071439 thatDET 9 12 later that afternoon
1013076350897071439 thatDET 2 5 mean that.
1013076350897071439 thatDET 23 26 “That’s
1013076350897071439 thatDET 55 58 knock that white
1013076350897071439 thatDET 4 7 before that happened
1013076350897071439 thatDET 8 11 and that sufficed
1013076350

...

1013076350897071439 thatDET 2 5 me that,
1013076350897071439 thatDET 7 10 like that.
1013076350897071439 thatDET 18 21 separate that way
1013076350897071439 thatDET 7 10 know that.
1013076350897071439 thatDET 0 3 —Isn’t that wonderful
1013076350897071439 thatDET 2 5 hear that trombone
1013076350897071439 thatDET 6 9 in that shifting
1013076350897071439 thatDET 12 15 since that day
1013076350897071439 thatDET 0 3 until that one
1013076350897071439 thatDET 14 17 like that.
1013076350897071439 thatDET 20 23 
that of
1013076350897071439 thatDET 30 33 by that boy
1013076350897071439 thatDET 7 10 of that which
1013076350897071439 thatDET 3 6 with that lovely
1013076350897071439 thatDET 11 14 , that silent
1013076350897071439 thatDET 4 7 of that flesh
1013076350897071439 thatDET 13 16 of that old
1013076350897071439 thatDET 57 60 like that of
1013076350897071439 thatDET 33 36 like that of
1013076350897071439 thatDET 6 9 hate that coati
1013076350897071439 thatDET 3 6 if that woman
10130763508

1013076350897071439 thatDET 23 26 for that gang
1013076350897071439 thatDET 20 23 with that form
1013076350897071439 thatDET 45 48 contemplate that child
1013076350897071439 thatDET 33 36 on that day
1013076350897071439 thatDET 31 34 , that

1013076350897071439 thatDET 0 3 At that very
1013076350897071439 thatDET 0 3 At that time
1013076350897071439 thatDET 25 28 in that crucial
1013076350897071439 thatDET 14 17 attribute that misfortune
1013076350897071439 thatDET 6 9 here that

1013076350897071439 thatDET 10 13 if that ever
1013076350897071439 thatDET 21 24 from that ritual
1013076350897071439 thatDET 0 3 On that night
1013076350897071439 thatDET 7 10 for that ridiculous
1013076350897071439 thatDET 9 12 about that brother
1013076350897071439 thatDET 21 24 to that produced
1013076350897071439 thatDET 47 50 like that—
1013076350897071439 thatDET 7 10 of that long
1013076350897071439 thatDET 32 35 of that girl
1013076350897071439 thatDET 29 32 that that man
1013076350897071439 thatDET 1

1013076350897071439 thatDET 22 25 behind that sort
1013076350897071439 thatDET 28 31 , that darkening
1013076350897071439 thatDET 13 16 of that girl
1013076350897071439 thatDET 6 9 repeat that?
1013076350897071439 thatDET 12 15 ’ That is
1013076350897071439 thatDET 3 6 late that same
1013076350897071439 thatDET 63 66 like that of
1013076350897071439 thatDET 9 12 in that stuff
1013076350897071439 thatDET 22 25 but that is
1013076350897071439 thatDET 22 25 ) that something
1013076350897071439 thatDET 57 60 like that—
1013076350897071439 thatDET 22 25 like that of
1013076350897071439 thatDET 5 8 of that room
1013076350897071439 thatDET 13 16 exactly that)
1013076350897071439 thatDET 3 6 —that of
1013076350897071439 thatDET 44 47 ) that sounds
1013076350897071439 thatDET 62 65 : that is
1013076350897071439 thatDET 5 8 all that agitation
1013076350897071439 thatDET 138 141 provoked that slight
1013076350897071439 thatDET 0 3 In that first
1013076350897071439 thatDET 13 16 , that appearance


1013076350897071439 thatDET 21 24 like that,
1013076350897071439 thatDET 9 12 of that,
1013076350897071439 thatDET 1 4 like that,
1013076350897071439 thatDET 16 19 Fuck that shit
1013076350897071439 thatDET 7 10 “That’s
1013076350897071439 thatDET 10 13 all that other
1013076350897071439 thatDET 29 32 But that night
1013076350897071439 thatDET 45 48 special that evening
1013076350897071439 thatDET 25 28 for that young
1013076350897071439 thatDET 2 5 , that night
1013076350897071439 thatDET 5 8 order that night
1013076350897071439 thatDET 2 5 of that threat
1013076350897071439 thatDET 3 6 do that to
1013076350897071439 thatDET 0 3 “That smell
1013076350897071439 thatDET 16 19 of that stink
1013076350897071439 thatDET 9 12 —that is
1013076350897071439 thatDET 5 8 , that faggot
1013076350897071439 thatDET 5 8 wants that killer
1013076350897071439 thatDET 24 27 from that time
1013076350897071439 thatDET 10 13 and that faggot
1013076350897071439 thatDET 3 6 you that to
1013076350897071439 t

1013076350897071439 thatDET 16 19 time that day
1013076350897071439 thatDET 35 38 think that was
1013076350897071439 thatDET 188 191 only that was
1013076350897071439 thatDET 5 8 

That was
1013076350897071439 thatDET 3 6 , that kind
1013076350897071439 thatDET 6 9 from that same
1013076350897071439 thatDET 0 3 And that was
1013076350897071439 thatDET 17 20 all that time
1013076350897071439 thatDET 9 12 : that was
1013076350897071439 thatDET 10 13 

That is
1013076350897071439 thatDET 4 7 of that solemn
1013076350897071439 thatDET 1 4 , that silvery
1013076350897071439 thatDET 1 4 enjoyable that room
1013076350897071439 thatDET 1 4 how that huge
1013076350897071439 thatDET 5 8 on that narrow
1013076350897071439 thatDET 10 13 isn’t that true
1013076350897071439 thatDET 20 23 like that,
1013076350897071439 thatDET 39 42 of that summer
1013076350897071439 thatDET 16 19 in that hospitable
1013076350897071439 thatDET 0 3 And that was
1013076350897071439 thatDET 24 27 along that street
10130

In [112]:
#that (DET)

print('that (DET)')
print('e.g. half that amount, that language')

#ru
oddsratio, pvalue = scipy.stats.fisher_exact([[561, 158404], [652, 158313]])
print('ru')
print(pvalue)

#pt
oddsratio, pvalue = scipy.stats.fisher_exact([[310, 68749], [283, 68776]])
print('pt')
print(pvalue)

#it
oddsratio, pvalue = scipy.stats.fisher_exact([[249, 72668], [299, 72618]])
print('it')
print(pvalue)

#es
oddsratio, pvalue = scipy.stats.fisher_exact([[312, 71066], [293, 71085]])
print('es')
print(pvalue)

that (DET)
e.g. half that amount, that language
ru
0.009595670402328539
pt
0.28461356664981147
it
0.03588764036732673
es
0.46336744321472134


### 4.8 X's own construction
Russian has a specific word for "one's own"; we wondered if that construction in English might be more frequent as a result.

In [10]:
#"PRP$'s own'"
pattern = [{"TAG": "PRP$"}, {"orth": "own"}]


matcher.add("own", None, pattern)
for filename in os.listdir(sourcefiledirectory):
    if filename.startswith("all"):
        
        f = open(filename, 'r')
        text = f.read()
        lines = sent_tokenize(text)
        outname = filename.replace('all', 'tmp_')
        with open(outname, 'w') as out:
            print(filename)
            for index,line in enumerate(lines):
                annotated_line = nlp(line)
                matches = matcher(annotated_line)
                for match_id, start, end in matches:
                    string_id = nlp.vocab.strings[match_id]  # Get string representation
                    span = annotated_line[start:end]  # The matched span
                    print(match_id, string_id, start, end, span.text)
                    out.write(str(match_id))
                    out.write("\n")
for filename in os.listdir(sourcefiledirectory):
    if filename.startswith("tmp_"):
        with open(filename, 'r') as f:
            outname = filename.replace('tmp_', 'PRP_own_')
            num_lines = 0
            with open(outname, 'w') as out:
                for line in f:
                    num_lines += 1
                print(filename)
                print(num_lines)
                out.write(str(num_lines))
matcher.remove("own")

all_ru.txt
2646123857127441627 own 8 10 my own
2646123857127441627 own 31 33 my own
2646123857127441627 own 13 15 his own
2646123857127441627 own 25 27 its own
2646123857127441627 own 36 38 my own
2646123857127441627 own 16 18 his own
2646123857127441627 own 15 17 his own
2646123857127441627 own 14 16 his own
2646123857127441627 own 4 6 his own
2646123857127441627 own 24 26 her own
2646123857127441627 own 4 6 her own
2646123857127441627 own 10 12 her own
2646123857127441627 own 2 4 your own
2646123857127441627 own 5 7 your own
2646123857127441627 own 6 8 my own
2646123857127441627 own 5 7 her own
2646123857127441627 own 34 36 their own
2646123857127441627 own 19 21 their own
2646123857127441627 own 2 4 his own
2646123857127441627 own 7 9 his own
2646123857127441627 own 32 34 his own
2646123857127441627 own 4 6 her own
2646123857127441627 own 11 13 her own
2646123857127441627 own 14 16 his own
2646123857127441627 own 11 13 his own
2646123857127441627 own 16 18 his own
264612385712744162

2646123857127441627 own 4 6 its own
2646123857127441627 own 16 18 her own
2646123857127441627 own 4 6 their own
2646123857127441627 own 4 6 his own
2646123857127441627 own 13 15 my own
2646123857127441627 own 42 44 their own
2646123857127441627 own 14 16 his own
2646123857127441627 own 16 18 my own
2646123857127441627 own 14 16 its own
2646123857127441627 own 12 14 their own
2646123857127441627 own 18 20 his own
2646123857127441627 own 33 35 his own
2646123857127441627 own 14 16 her own
2646123857127441627 own 32 34 its own
2646123857127441627 own 48 50 my own
2646123857127441627 own 22 24 his own
2646123857127441627 own 16 18 my own
2646123857127441627 own 3 5 my own
all_it.txt
2646123857127441627 own 3 5 his own
2646123857127441627 own 32 34 his own
2646123857127441627 own 57 59 its own
2646123857127441627 own 48 50 our own
2646123857127441627 own 5 7 my own
2646123857127441627 own 15 17 my own
2646123857127441627 own 9 11 my own
2646123857127441627 own 13 15 its own
2646123857127441

In [11]:
#One's own

#ru
print('determiner + own')
print('e.g. his own')
oddsratio, pvalue = scipy.stats.fisher_exact([[148, 79335], [101, 79382]])
print('ru')
print(pvalue)

#pt
oddsratio, pvalue = scipy.stats.fisher_exact([[22, 34508], [44, 34486]])
print('pt')
print(pvalue)

#it
oddsratio, pvalue = scipy.stats.fisher_exact([[37, 36422], [46, 36412]])
print('it')
print(pvalue)

#es
oddsratio, pvalue = scipy.stats.fisher_exact([[37, 35652], [45, 35644]])
print('es')
print(pvalue)

determiner + own
e.g. his own
ru
0.003446580302803858
pt
0.009177483909881105
it
0.32585477608440927
es
0.4394417580376581
